# **Project OPEN AI - Jukebox**

### Membros / Members

* Antônio Barros
* André Vasconcelos
* André Ferreira
* Heitor Santos
* João Vítor Valadares
* Marcos Prysthon
* Robson Oliveira

Para a realização dessa ferramenta utilizamos de base dois colabs: o disponibilizado oficialmente pelo [Open AI](https://colab.research.google.com/github/openai/jukebox/blob/master/jukebox/Interacting_with_Jukebox.ipynb) e uma [Versão](https://colab.research.google.com/drive/1zBjIfvP2p90HDaldL6_tcxcwpExuEnof#scrollTo=ZPdMgaH_BPGN) que demonstra como utilizar músicas como entrada para o modelo.

NOTA IMPORTANTE SOBRE OS REQUISITOS DO SISTEMA:

Se você estiver se conectando a um runtime hospedado, certifique-se de que ele tenha uma GPU P100 (opcionalmente execute! Nvidia-smi para confirmar). Vá para Editar> Configurações do Notebook para definir isso.

O CoLab pode primeiro atribuir a você uma máquina de memória inferior se você estiver usando um tempo de execução hospedado. Nesse caso, a primeira vez que você tentar carregar o modelo 5B, ele ficará sem memória e, em seguida, você será solicitado a reiniciar com mais memória (em seguida, retorne ao topo deste CoLab). Se você continuar a ter problemas de memória depois disso (ou tiver problemas em sua própria configuração doméstica), mude para o modelo 1B.

Se você estiver usando uma GPU local, recomendamos V100 ou P100 com memória de GPU de 16 GB para melhor desempenho. Para GPUs com menos memória, recomendamos o uso do modelo 1B e um tamanho de lote menor.

-----
To make this tool, we used two google colab notebooks as references: the oficial one, made available by [Open AI](https://colab.research.google.com/github/openai/jukebox/blob/master/jukebox/Interacting_with_Jukebox.ipynb) and another [Version](https://colab.research.google.com/drive/1zBjIfvP2p90HDaldL6_tcxcwpExuEnof#scrollTo=ZPdMgaH_BPGN) that shows how to use songs as an input to the model.

IMPORTANT NOTE ON SYSTEM REQUIREMENTS:

If you are connecting to a hosted runtime, make sure it has a P100 GPU (optionally run !nvidia-smi to confirm). Go to Edit>Notebook Settings to set this.

CoLab may first assign you a lower memory machine if you are using a hosted runtime.  If so, the first time you try to load the 5B model, it will run out of memory, and then you'll be prompted to restart with more memory (then return to the top of this CoLab).  If you continue to have memory issues after this (or run into issues on your own home setup), switch to the 1B model.

If you are using a local GPU, we recommend V100 or P100 with 16GB GPU memory for best performance. For GPU’s with less memory, we recommend using the 1B model and a smaller batch size throughout.  



## Inicializando o Ambiente

A seguir vamos inicializar o ambiente, para deixar já pronto para a utilização ou renicialização do ambiente. Vamos montar o drive para salvar as músicas geradas, verificar as informações da placa da NVIDIA, baixar as depedências de projeto e atualizar as fotos dos artistas.

-----

##Setting the enviroment up

Now we're going to set our enviroment up, letting it ready to be used or reset. Let's mount our drive to save the generated songs, check NVIDIA graphics card information, download the project dependecies and upgrade the artists photos.

In [ ]:
#@title Monte o seu drive aqui / Mount your drive here { display-mode: "form" }
from google.colab import drive, widgets
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title Verifique as informações da placa Nvidia, pacotes e atualize as fotos dos artistas / Check the NVIDIA card informations, packages and artists photos { display-mode: "form" }
!nvidia-smi
!pip install git+https://github.com/openai/jukebox.git
!gdown --id 192GcOUskBo7-ILC1JtVhRvpMF7dIsohu
!unrar x 'photos.rar'

Mon Dec 13 18:33:14 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#@title Prepare o ambiente. / Prepare the environment. { display-mode: "form" }
import jukebox
import torch as t
import librosa
import ipywidgets as widgets
from IPython.display import display
import os
import IPython
from IPython.display import Audio
import IPython
import google.colab
from jukebox.make_models import make_vqvae, make_prior, MODELS, make_model
from jukebox.hparams import Hyperparams, setup_hparams
from jukebox.sample import sample_single_window, _sample, \
                           sample_partial_window, upsample, \
                           load_prompts
from jukebox.utils.dist_utils import setup_dist_from_mpi
from jukebox.utils.torch_utils import empty_cache
rank, local_rank, device = setup_dist_from_mpi()
from google.colab import files

# Compondo utilizando 5B ou 1B Modelos



Para obter mais controle sobre as gerações, tente compor com os modelos de letras 5B ou 1B. Novamente, especifique qual a sua pasta do drive que vai estar sendo utilizada, a frequência da sua amostra, seu artista, gênero e letras. Porém, agora em vez de gerar a amostra inteira, o modelo retornará 3 opções curtas para a abertura da peça (ou até 16 opções se você usar o modelo 1B). Escolha o seu favorito e continue o loop pelo tempo que desejar. Ao longo dessas etapas, você ouvirá o áudio no nível anterior superior, o que significa que terá um som muito barulhento. Quando estiver satisfeito com sua co-criação, continue na seção de upsampling. Isso renderizará a peça em qualidade de áudio superior.

NOTA: O CoLab primeiro atribuirá a você uma máquina de memória inferior se você estiver usando um tempo de execução hospedado. A próxima célula ficará sem memória e você será solicitado a reiniciar com mais memória (em seguida, retornar ao topo deste CoLab). Se você continuar a ter problemas de memória depois disso (ou tiver problemas em sua própria configuração), mude para o modelo 1B.

NOTA: Se não especificar nenhuma pasta para entrada do modelo, ele vai criar uma nova ao termino da execução desse notebook.

----
#Composing using 5B or 1B models
For more control over the generations, try co-composing with either the 5B or 1B Lyrics Models.  Again, specify your drive folder, sample rate, artist, genre, and lyrics. However, now instead of generating the entire sample, the model will return 3 short options for the opening of the piece (or up to 16 options if you use the 1B model instead).  Choose your favorite, and then continue the loop, for as long as you like.  Throughout these steps, you'll be listening to the audio at the top prior level, which means it will sound quite noisy.  When you are satisfied with your co-creation, continue on through the upsampling section. This will render the piece in higher audio quality.

NOTE: CoLab will first assign you a lower memory machine if you are using a hosted runtime.  The next cell will run out of memory, and then you'll be prompted to restart with more memory (then return to the top of this CoLab).  If you continue to have memory issues after this (or run into issues on your own home setup), switch to the 1B model. 

NOTE: If you don't specify any folder for input, it will create a new one at the end of running this notebook.

In [ ]:
#@markdown Escolha o modelo, o sample rate (recomendamos o 44100) e a pasta do drive em que os samples serão salvos. { display-mode: "form" }

#@markdown -----
#@markdown Choose the model, sample rate (we recommend 44100) and the drive folder in which the samples will be saved.
model = "1b_lyrics" #@param ['1b_lyrics','5b_lyrics'] 
hps = Hyperparams()
sample_rate= 44100 #@param [44100, 48000, 32000,22050, 11025, 8000]
hps.sr = sample_rate
hps.n_samples = 3 if model in ('5b', '5b_lyrics') else 16
# Specifies the directory to save the sample in.
# We set this to the Google Drive mount point.
pasta = 'result_mus'  #@param {type:"string"}
hps.name = '/content/drive/MyDrive/'+pasta
hps.sample_length = 1048576 if model in ('5b', '5b_lyrics') else 786432 
chunk_size = 16 if model in ('5b', '5b_lyrics') else 32
max_batch_size = 3 if model in ('5b', '5b_lyrics') else 16
hps.hop_fraction = [.5, .5, .125] 
hps.levels = 3

vqvae, *priors = MODELS[model]
vqvae = make_vqvae(setup_hparams(vqvae, dict(sample_length = hps.sample_length)), device)
top_prior = make_prior(setup_hparams(priors[-1], dict()), vqvae, device)

NameError: ignored

# Selecionar Modo do Modelo
Escolha entre o modo "Ancestral" ou o "Primed".



*   **Ancestral**: modo onde apenas utilizamos os parâmetros de texto e númericos do modelo para gerar a saída, como artista, gênero, letra da música, nível da temperatura, entre outros.
*   **Primed**: modo onde além de utilizarmos parâmetros de texto e númericos do modelo para gerar a saída, como no "Ancestral", a inteligência vai se basear principalmente no arquivo de audio que for transferido a ele.

-----

# Select model mode

*   **Ancestral**: mode in which we only user text and numerical parameters to generate the output, like artist, genre, lyrics, temperature level, and others.
*   **Primed**: mode in which beyond the text and numerical parameters, the AI will be using mostly use the audio file uploaded as an reference to generate the output.

In [ ]:
# The default mode of operation.
# Creates songs based on artist and genre conditioning.
mode = 'ancestral' #@param ['ancestral','primed']
codes_file=None

In [ ]:
#@markdown Escolha o arquivo de música, caso for o modo primed { display-mode: "form" }

#@markdown -----
#@markdown Choose the audio file, in case you chose the primed mode
if mode == 'primed':
  uploaded = files.upload()
  for fn in uploaded.keys():
    audio_file = '/content/'+fn
  prompt_length_in_seconds=12
else:
  audio_file=None
  prompt_length_in_seconds=None

In [ ]:
#@markdown O arquivo da música é o seguinte:


#@markdown -----
#@markdown The audio file is this one:
if mode == 'primed':
  print(audio_file)
  Audio(audio_file)
else:
  print('Você escolheu o modo ancestral, esse modo não tem arquivo de música, pode continuar')

Faça aqui as configurações das suas amostras, como nome do artista, gênero e letra, lembrando que toda vez que essa célula for executada, a foto do artista na saída dele vai ser atualizado (Se houver modificação no artista), faça isso para além de se familiarizar um pouco com o artista, para reconhecer se os artistas são os mesmos.

NOTA: Lembrando que o "sample_length_in_seconds" é sobre o tamanho da sua **amostra final** em segundos.

-----
Set up your samples here, you can choose artist, genre and lyrics, notice that every time this cell is run, the artist photo in the output will be updated (if the artist changes), do it to get familiar with the artist and to see if the artist matches with the one you're thinking of.

NOTE: remember that "sample_length_in_seconds" is the length of your **final sample** in seconds.

In [ ]:
#@title Faça configurações dos samples / Set your samples up:
sample_hps = Hyperparams(dict(mode=mode, codes_file=codes_file, audio_file=audio_file, prompt_length_in_seconds=prompt_length_in_seconds))
sample_length_in_seconds = 30 #@param {"type":"slider",min:30,max:240}
hps.sample_length = (int(sample_length_in_seconds*hps.sr)//top_prior.raw_to_tokens)*top_prior.raw_to_tokens
assert hps.sample_length >= top_prior.n_ctx*top_prior.raw_to_tokens, f'Please choose a larger sampling rate'
artist = "the police" #@param ["!!!", "!distain", "(spunge)", "*nsync", "...and oceans", ".moneen.", "10 years", "108", "10cc", "112", "116 clique", "12 rods", "1208", "1349", "13th floor elevators", "16", "16 volt", "1910 fruitgum company", "2 brothers on the 4th floor", "2 chainz", "2 live crew", "2 skinnee j's", "2 unlimited", "21 guns", "21 savage", "220 volt", "25 ta life", "28 days", "2nd chapter of acts", "2pac", "3 feet smaller", "3 inches of blood", "30 odd foot of grunts", "36 crazyfists", "38 special", "3lw", "3rd bass", "3t", "4 strings", "4him", "4minute", "5 chinese brothers", "5 seconds of summer", "50 cent", "500 miles to memphis", "504 boyz", "54-40", "59 times the pain", "69 boyz", "6ix9ine", "6lack", "7 seconds", "7 year bitch", "702", "764-hero", "7l & esoteric", "7th cycle", "8 foot sativa", "88 fingers louie", "8ball", "8ball & mjg", "98 mute", "98°", "999", "a bullet for pretty boy", "a change of pace", "a day to remember", "a fine frenzy", "a flock of seagulls", "a global threat", "a great big world", "a halo called fred", "a hill to die upon", "a house", "a life divided", "a life once lost", "a loss for words", "a lot like birds", "a plea for purging", "a silent film", "a split-second", "a static lullaby", "a storm of light", "a tortured soul", "a tribe called quest", "a whisper in the noise", "a$ap ferg", "a$ap mob", "a$ap rocky", "a*teens", "a. l. lloyd", "a.a. bondy", "a.c. newman", "a.c.t", "a1", "aage kvalbein", "aaradhna", "aaron carter", "aaron copland", "aaron lee tasjan", "aaron lewis", "aaron lines", "aaron neville", "aaron shust", "aaron sprinkle", "aaron tippin", "aaron watson", "ab-soul", "abandon all ships", "abandoned pools", "abba", "abbey lincoln", "abbie gale", "abc", "aberfeldy", "abgott", "abigail washburn", "abigail williams", "abigor", "abk", "ablaze my sorrow", "abney park", "above & beyond", "abra moore", "absolution project", "abysmal dawn", "ac/dc", "acappella", "accept", "accessory", "ace enders & a million different people", "ace frehley", "ace of base", "ace troubleshooter", "acid bath", "acid drinkers", "acid house kings", "acid king", "acid ranch", "acid reign", "acid witch", "acoustic junction", "across the border", "action action", "action adventure world", "active child", "acumen nation", "ad", "adagio", "adam ant", "adam cohen", "adam green", "adam lambert", "adam sandler", "adele", "adem", "adestria", "adhesive", "adolf busch", "ador dorath", "adore delano", "adrenaline mob", "adrian belew", "adriano celentano", "adrienne young", "adult.", "advance base", "aeon", "aeon zen", "aerosmith", "aesma daeva", "aesop rock", "aesthetic perfection", "aeternus", "afroman", "after 7", "after all", "after forever", "after the burial", "after the fire", "afu-ra", "against all authority", "against me!", "against the current", "agalloch", "agathocles", "agathodaimon", "agent 51", "agents of mercy", "agnes", "agnes obel", "agnetha fältskog", "agnostic front", "agonoize", "agoraphobic nosebleed", "agressor", "aiden", "aimee mann", "aion", "air", "air supply", "airbourne", "akala", "akcent", "akinyele", "akissforjersey", "akon", "akrobatik", "akron family", "al axy", "al b. sure!", "al denson", "al green", "al jarreau", "al kooper", "al martino", "al stewart", "alabama", "alabama 3", "alabama shakes", "alabama thunderpussy", "alan hull", "alan jackson", "alan parsons", "alan stivell", "alannah myles", "alaska thunderfuck", "albert collins", "albert hammond", "albert hammond", "albert king", "albert west", "alberta cross", "alborosie", "alcatrazz", "alcazar", "aldo nova", "alejandro escovedo", "alela diane", "alesana", "alesha dixon", "alessandro scarlatti", "alessi brothers", "alessia cara", "alestorm", "alex harvey", "alex lloyd", "alex story", "alexander borodin", "alexander scriabin", "alexandra burke", "alexis korner", "alexis taylor", "alexisonfire", "alexz johnson", "alfred cortot", "ali project", "alice cooper", "alice in chains", "alicia de larrocha", "alicia keys", "alien ant farm", "alien sex fiend", "alina simone", "alison krauss", "alison krauss & union station", "alison moyet", "alison wonderland", "alkaline trio", "all", "all saints", "all shall perish", "all that remains", "all them witches", "all-4-one", "allan taylor", "allen toussaint", "allie x", "allies", "allison iraheta", "allison moorer", "allister", "allo darlin'", "allstar weekend", "allure", "almah", "almôra", "aloe blacc", "aloha", "alpha blondy", "alphabeat", "alphaville", "alt-j", "altan", "altar boys", "altaria", "alter bridge", "altered images", "alvin lee", "alvin stardust", "alvin youngblood hart", "aly & aj", "alyson stoner", "am & shawn lee", "amanda jenssen", "amanda lear", "amanda marshall", "amanda perez", "amaranthe", "amazing blondel", "amber", "amber pacific", "amberian dawn", "ambrosia", "amduscia", "amebix", "amel larrieux", "amen", "amenra", "america", "american aquarium", "american authors", "american head charge", "american juniors", "american me", "american pleasure club", "amerie", "amir obè", "amon amarth", "amon düül ii", "amorphis", "amos lee", "amy grant", "amy holland", "amy macdonald", "amy ray", "amy rigby", "amy shark", "amy winehouse", "an angle", "an cafe", "an horse", "ana popovic", "anacrusis", "anarbor", "anastacia", "anata", "anathallo", "anathema", "anaïs mitchell", "anchor", "ancient", "ancient bards", "ancient rites", "and also the trees", "and loathing in las vegas", "and one", "anders johansson", "anders manga", "anders osborne", "anderson .paak", "andi deris", "andr previn", "andr s schiff", "andr watts", "andraé crouch", "andre matos", "andre nickatina & equipto", "andrea bocelli", "andrea schroeder", "andreas johnson", "andrew belle", "andrew gold", "andrew huang", "andrew lloyd webber", "andrew peterson", "andrew w.k.", "andreya triana", "android lust", "andromeda", "andru donalds", "andy davis", "andy griggs", "andy m. stewart", "andy mineo", "andy park", "andy partridge", "andy stochansky", "ane brun", "anekdoten", "angel", "angel corpse", "angel dust", "angela bofill", "angela mccluskey", "angelic upstarts", "angelina", "angelo branduardi", "angelo de augustine", "angels & agony", "angelspit", "angelus apatrida", "angie stone", "angra", "angry samoans", "angus & julia stone", "animal collective", "animal liberation orchestra", "animal logic", "animaniacs", "animosity", "anita baker", "anita carter", "ann beretta", "ann wilson", "anna ternheim", "anna tivel", "anna tsuchiya", "anne akiko meyers", "anne clark", "anne murray", "anne sophie mutter", "anner bylsma", "annette hanshaw", "anni b sweet", "annie", "annie herring", "annie lennox", "annihilator", "annuals", "anointed", "anorexia nervosa", "ant & dec", "antaeus", "antagonist a.d.", "antestor", "anthony evans", "anthony green", "anthony hamilton", "anthony phillips", "anthrax", "anti-depressive delivery", "anti-flag", "anti-nowhere league", "antimatter", "antique", "antiskeptic", "anton bruckner", "anton n dvo k", "antonio meneses", "antonio vivaldi", "antony and the johnsons", "anubis", "anvil", "any given day", "anya marina", "anything box", "anúna", "aorta", "apache indian", "apartment 26", "aphex twin", "aphrodite's child", "apocalypse hoboken", "apocalyptica", "apollo sunshine", "apologetix", "apostasy", "april wine", "aqua", "aqualung", "aqueduct", "arabesque", "aram khachaturian", "arcade fire", "arcadi volodos", "arcana", "arcane", "arcangelo corelli", "architects", "architecture in helsinki", "archive", "arcturus", "arena", "aretha franklin", "argent", "arghoslent", "ari hest", "ariana grande", "ariel pink", "arjen anthony lucassen", "ark", "arlo guthrie", "armageddon", "armand van helden", "armin van buuren", "armored saint", "army of freshmen", "army of lovers", "army of the pharaohs", "arno", "arnold schoenberg", "arrested development", "arsis", "arsonists", "arsonists get all the girls", "art bears", "art garfunkel", "art of dying", "artension", "arthur beatrice", "arthur crudup", "arthur grumiaux", "arthur rubinstein", "artie shaw", "artifacts", "artillery", "artrosis", "artur schnabel", "arturo benedetti michelangeli", "as blood runs black", "as cities burn", "as i lay dying", "as we fight", "asaf avidan", "asaf avidan & the mojos", "aselin debison", "asg", "asha", "ashanti", "ashbury heights", "asher roth", "ashes you leave", "ashford & simpson", "ashland high", "ashlee simpson", "ashley monroe", "ashley tisdale", "ashton shepherd", "asia", "asian dub foundation", "asking alexandria", "asleep at the wheel", "asobi seksu", "asp", "asphyx", "ass ponys", "assassin", "assemblage 23", "assuming we survive", "assück", "astarte", "astronautalis", "astrud gilberto", "aswad", "at the drive-in", "at the gates", "atari teenage riot", "atb", "atc", "atlanta rhythm section", "atlas sound", "atmosphere", "atomic kitten", "atomic opera", "atomic rooster", "atreyu", "atrocity", "attack attack!", "attack in black", "attila", "attrition", "au revoir simone", "audience", "audio adrenaline", "audra mcdonald", "audrey", "audrey assad", "augie march", "august alsina", "august burns red", "augustana", "aura dione", "aura noir", "austin mahone", "austra", "australian crawl", "austrian death machine", "autopilot off", "autopsy", "autumn", "autumnblaze", "avalanche city", "avantasia", "avatar", "avenged sevenfold", "averi", "aversions crown", "aviators", "avicii", "avulsed", "axe", "axel rudi pell", "axenstar", "axxis", "aynsley lister", "ayreon", "ayria", "aythis", "ayọ", "az", "azealia banks", "aztec camera", "azure ray", "aïboforcen", "b la bart k", "b! machine", "b'z", "b*witched", "b-legit", "b.b. king", "b.j. thomas", "b.o.b", "b.o.b.", "b3", "b5", "baba brinkman", "babbie mason", "babe ruth", "babes in toyland", "baby bash", "babybird", "babyface", "babyland", "babylon whores", "babyshambles", "baccara", "bachman-turner overdrive", "backseat goodbye", "backstreet boys", "backyard babies", "bad astronaut", "bad bones", "bad books", "bad boys blue", "bad brains", "bad company", "bad habit", "bad manners", "bad religion", "badfinger", "badlands", "badly drawn boy", "baha men", "bahamas", "bal-sagoth", "balance of power", "balance problems", "bald vulture", "balkan beat box", "ballyhoo!", "balto", "balzac", "bananafishbones", "bananarama", "band of horses", "band of susans", "bane", "bang gang", "barbara mandrell", "barbara mason", "barbarossa", "barbecue bob", "barbra streisand", "barclay james harvest", "barefoot truth", "barney", "barrio boyzz", "barry adamson", "barry louis polisar", "barry mcguire", "barry white", "barstool prophets", "bart davenport", "barzin", "bascom lamar lunsford", "basement jaxx", "basia", "basia bulat", "bass drum of death", "basshunter", "bastro", "bat for lashes", "bathory", "baths", "batmobile", "baton rouge", "battery", "battle beast", "battlelore", "bauhaus", "baxter", "bay city rollers", "bazzi", "bbmak", "be bop deluxe", "be your own pet", "be'lakor", "beach fossils", "beach house", "beach slang", "beachwood sparks", "beady belle", "beady eye", "bear in heaven", "bear vs. shark", "bear's den", "beardfish", "bearstronaut", "beartooth", "beast in black", "beastie boys", "beat crusaders", "beat farmers", "beat happening", "beatnik termites", "beau", "beautiful eulogy", "beaux arts trio", "beaver", "bebe & cece winans", "bebe winans", "bebel gilberto", "bebo norman", "beborn beton", "beck", "becoming the archetype", "bed ich smetana", "bedhead", "bedlight for blue eyes", "bee gees", "before their eyes", "beggars & thieves", "beggars opera", "behemoth", "behexen", "beholder", "beirut", "believer", "belinda carlisle", "bell book & candle", "bell x1", "bella hardy", "bella morte", "belle and sebastian", "belleruche", "belouis some", "beloved", "belphegor", "belvedere", "ben caplan", "ben e. king", "ben kweller", "ben lee", "ben moody", "ben taylor", "ben weasel", "beneath the massacre", "beneath the sky", "beng beng cocktail", "benjamin britten", "benjamin francis leftwich", "benjamin gibbard", "benno moiseiwitsch", "benny benassi", "benny goodman", "benny hester", "benny mardones", "benny sings", "bent knee", "bentley jones", "beracah", "beres hammond", "berlin", "berman", "bernard butler", "berried alive", "bert jansch", "bert williams", "besatt", "beseech", "bessie smith", "beth hart", "beth hart & joe bonamassa", "beth hirsch", "beth nielsen chapman", "beth orton", "bethany dillon", "bethel music", "beto vázquez infinity", "betraying the martyrs", "bette midler", "better luck next time", "betty who", "bettye lavette", "between the buried and me", "between the trees", "beulah", "beyoncé", "beyond dawn", "beyond the black", "bic runga", "big & rich", "big audio dynamite", "big bad voodoo daddy", "big bang", "big big train", "big bill broonzy", "big black", "big boi", "big business", "big d and the kids table", "big daddy", "big ed", "big joe turner", "big joe williams", "big k.r.i.t.", "big l", "big maceo", "big maybelle", "big moe", "big mountain", "big pokey", "big sean", "big star", "big tent revival", "big thief", "big time rush", "big tymers", "big wreck", "biga ranx", "bigwig", "bikini kill", "bile", "bill anderson", "bill fay", "bill mallonee", "bill monroe", "bill nelson", "bill ward", "bill withers", "bill wyman's rhythm kings", "billie eilish", "billie holiday", "billie jo spears", "billie piper", "billie ray martin", "billie the vision & the dancers", "billy boy arnold", "billy bragg", "billy bragg & wilco", "billy crawford", "billy currington", "billy gilman", "billy idol", "billy joe royal", "billy joe shaver", "billy joel", "billy ocean", "billy preston", "billy ray cyrus", "billy sprague", "billy squier", "billy thorpe", "billy walker", "binary star", "bing crosby", "binoculers", "binärpilot", "biohazard", "birdman", "birds in row", "birdy", "birmingham 6", "birth control", "bishop allen", "bitch", "biz markie", "bizzy bone", "björk", "björn ulvaeus & benny andersson", "black", "black 47", "black bomb a", "black breath", "black country communion", "black flag", "black francis", "black grape", "black knights", "black label society", "black light burns", "black moth super rainbow", "black mountain", "black sabbath", "black sheep", "black star", "black star riders", "black stone cherry", "black tide", "black tusk", "black uhuru", "black veil brides", "blackfoot", "blackguard", "blackie and the rodeo kings", "blackjack", "blacklisted", "blackmore's night", "blackrain", "blackthorn", "blaine larsen", "blake shelton", "blank & jones", "blanks 77", "blaqk audio", "blaque", "blaze bayley", "blaze ya dead homie", "blazin' squad", "bleach", "bleed from within", "bleed the sky", "bleeding through", "blessed by a broken heart", "blessthefall", "blind blake", "blind boy fuller", "blind guardian", "blind lemon jefferson", "blind passengers", "blind pilot", "blind willie johnson", "blind willie mctell", "blind witness", "blindside", "blindspott", "bliss n eso", "blitz", "blitzkid", "bloc party", "blondie", "blood", "blood", "blood duster", "blood on the dance floor", "blood or whiskey", "blood orange", "blood oranges", "blood stain child", "blood tsunami", "bloodbath", "bloodgood", "bloodhound gang", "bloodlined calligraphy", "blowsight", "blu cantrell", "blue café", "blue cheer", "blue highway", "blue mountain", "blue october", "blue rodeo", "blue stahli", "blue system", "blue tears", "blue öyster cult", "bluehorses", "blueprint", "blues magoos", "blues pills", "blues traveler", "blutengel", "bo carter", "bo diddley", "boa", "boards of canada", "bob carlisle", "bob dylan", "bob geldof", "bob marley & the wailers", "bob moses", "bob rivers", "bob seger", "bobbie gentry", "bobby blue bland", "bobby brown", "bobby darin", "bobby goldsboro", "bobby helms", "bobby mcferrin", "bobby o", "bobby v", "bobby valentino", "bobby vee", "bobby vinton", "bobby womack", "bobo in white wooden houses", "bodies of water", "body count", "bodyfarm", "bodyjar", "bokka", "bolt thrower", "bomb the music industry!", "bombay bicycle club", "bombshell rocks", "bomfunk mc's", "bon iver", "bon jovi", "bonaparte", "bondage fairies", "bonded by blood", "bone thugs-n-harmony", "bones brigade", "boney james", "boney m.", "bonfire", "bongzilla", "bonnie bianco", "bonnie owens", "bonnie pink", "bonnie raitt", "bonnie tyler", "bonobo", "boogie down productions", "boomkat", "boondox", "bootsauce", "bootsy collins", "borknagar", "born against", "born from pain", "born of osiris", "born ruffians", "boss hogg outlawz", "boston", "bound for glory", "bound stems", "bow wow", "bow wow wow", "bowerbirds", "bowes & morley", "boxcar willie", "boy & bear", "boy george", "boy harsher", "boy meets girl", "boyce avenue", "boys night out", "boysetsfire", "boytronic", "boyz ii men", "boyzone", "boz scaggs", "bracket", "brad paisley", "brain drill", "brainiac", "brainpool", "brainstorm", "brand nubian", "brandi carlile", "brandon heath", "brandtson", "brandy", "brandy clark", "brave saint saturn", "brazilian girls", "brazzaville", "breach of trust", "bread", "breakdown of sanity", "breaking benjamin", "breaking laces", "breathe carolina", "brenda lee", "brenda russell", "brendan benson", "brendan perry", "brentalfloss", "brenton brown", "brenton wood", "bret michaels", "brett anderson", "brett dennen", "brett young", "brian doerksen", "brian eno", "brian hyland", "brian kennedy", "brian may", "brian mcfadden", "brian mcknight", "brian peters", "brian setzer", "brian wilson", "brick & lace", "bride", "bridgit mendler", "bright eyes", "bring me the horizon", "brisa roché", "british sea power", "britney spears", "britny fox", "broadway", "brockhampton", "brodka", "broken bones", "broken social scene", "brokencyde", "bronski beat", "bronze nazareth", "broods", "brook benton", "brooke fraser", "brooke white", "brooklyn bounce", "brooks & dunn", "bros", "brotha lynch hung", "brother cane", "brother dege", "brother firetribe", "brownie mcghee", "bruce cockburn", "bruce dickinson", "bruce hornsby", "bruce hornsby and the range", "bruce hungerford", "bruce mcculloch", "bruce springsteen", "bruno mars", "brutal truth", "brutality", "bryan adams", "bryan ferry", "bryan rice", "bt", "bts", "buck 65", "buck owens", "buck owens & susan raye", "buckcherry", "bucks fizz", "buddy & julie miller", "buddy guy", "buddy guy & junior wells", "buddy holly", "buddy jewell", "buddy miller", "buddy moss", "buddy wasisname and the other fellers", "budgie", "buffalo springfield", "buffy sainte-marie", "buggles", "building 429", "bukimina", "bukka white", "bulldozer", "bullet for my valentine", "bulletboys", "bullets and octane", "bumblefoot", "bun b", "bunny wailer", "burden of a day", "burl ives", "burlap to cashmere", "burn halo", "burning heads", "burning point", "burning witches", "burst", "burt bacharach", "burton cummings", "bury tomorrow", "bury your dead", "busdriver", "bushwick bill", "busta rhymes", "busted", "butterfly boucher", "buzzcocks", "by the tree", "byron cage", "børns", "c sar franck", "c-lekktor", "c-murder", "c.c. catch", "c.w. mccall", "c.w. stoneking", "cab calloway", "cactus", "cadillac blindside", "cady groves", "caedmon's call", "caesar", "caetano veloso", "cage", "caitlyn smith", "cake bake betty", "cal smith", "calabrese", "calexico", "caliban", "calibretto", "califone", "caligula's horse", "call the cops", "callisto", "calvin harris", "cam'ron", "camel", "cameo", "camera obscura", "camille", "camille saint sa ns", "can", "cancer", "cancer bats", "cancerslug", "candi staton", "candlemass", "candy butchers", "canibus", "cannabis corpse", "canned heat", "cannibal corpse", "cannonball statman", "capercaillie", "capital cities", "capital kings", "capital lights", "capitol steps", "capleton", "cappella", "captain beefheart and his magic band", "captain beefheart and the magic band", "captain beyond", "captain chunk!", "captain jack", "captain tractor", "capture the crown", "car seat headrest", "carach angren", "caravan", "carbon leaf", "carcass", "caribou", "carl maria von weber", "carl nielsen", "carl orff", "carl perkins", "carl wilson", "carla thomas", "carly rae jepsen", "carly simon", "carman", "carmen mcrae", "carnal forge", "carnifex", "carnival in coal", "caro emerald", "carole king", "carolina liar", "caroline herring", "carolyn arends", "carpark north", "carpathian forest", "carpenters", "carrie newcomer", "carrie underwood", "carson robison", "cartel", "carter the unstoppable sex machine", "cary brothers", "casanova", "cascada", "casey bill weldon", "casey donahew band", "cash cash", "cash rivers and the sinners", "casper & the cookies", "cass mccombs", "cassadee pope", "cassandra wilson", "cast", "castanets", "casting crowns", "casual", "cat power", "cat rapes dog", "cat stevens", "catamenia", "catch 22", "caterina valente", "catharsis", "cathedral", "catie curtis", "catman cohen", "cause & effect", "cause for effect", "cavalera conspiracy", "cave in", "cece winans", "cee lo green", "celestial season", "celesty", "cellar darling", "celldweller", "celluloide", "celph titled", "celtic frost", "celtic thunder", "celtic woman", "cemetary", "centinex", "centro-matic", "centvrion", "cephalic carnage", "cerebral fix", "ceremony", "chad brownlee", "chairlift", "chaka khan", "chamber - l'orchestre de chambre noir", "chameleon circuit", "chamillionaire", "champion jack dupree", "chandeen", "chantal kreviazuk", "chanté moore", "chaos uk", "charles aznavour", "charles bradley", "charles bronson", "charles harrison", "charles manson", "charley patton", "charley pride", "charli xcx", "charlie landsborough", "charlie louvin", "charlie musselwhite", "charlie parr", "charlie peacock", "charlie rich", "charlie sexton", "charlie simpson", "charlotte gainsbourg", "charlotte hatherley", "charlotte martin", "charly mcclain", "chase & status", "chase rice", "chasing victory", "chavez", "che'nelle", "cheap trick", "chef'special", "chelsea grin", "chelsea wolfe", "chemical vocation", "chemlab", "chenoa", "cher", "cher lloyd", "cheri dennis", "cherish the ladies", "cherrelle", "cherry ghost", "cherry glazerr", "cherry poppin' daddies", "cheryl cole", "chester watson", "chet atkins", "chevelle", "chic", "chicago", "chicane", "chickenfoot", "chicks on speed", "chico debarge", "chihiro onitsuka", "childish gambino", "children 18", "children of bodom", "chilliwack", "china", "china crisis", "chingo bling", "chiodos", "chipmunk", "chixdiggit!", "choclair", "choirboys", "choking victim", "chris august", "chris botti", "chris brown", "chris caffery", "chris cagle", "chris chameleon", "chris connor", "chris cornell", "chris de burgh", "chris garneau", "chris hillman", "chris hillman & herb pedersen", "chris isaak", "chris knight", "chris knox", "chris ledoux", "chris merritt", "chris mills", "chris murray", "chris norman", "chris rea", "chris spedding", "chris stapleton", "chris thompson", "chris tomlin", "chris webby", "chris young", "christ analogue", "christafari", "christian bautista", "christie", "christina aguilera", "christina grimmie", "christina milian", "christine and the queens", "christine fellows", "christine mcvie", "christon gray", "christoph eschenbach", "christoph willibald gluck", "christopher lee", "christy nockels", "chrom", "chromatics", "chromeo", "chubby checker", "chuck berry", "chuck ragan", "chunk! no", "chvrches", "ciara", "cibelle", "cibo matto", "cilla black", "cimorelli", "cinderella", "cinderella effect", "cindy bullens", "cinema bizarre", "circa survive", "circa waves", "circle ii circle", "circle jerks", "circle of dust", "circle takes the square", "circulatory system", "circus maximus", "circus of power", "cisco houston", "cissy houston", "city and colour", "civil twilight", "civil war", "cky", "clan of xymox", "clannad", "clap your hands say yeah", "clara haskil", "clara luzia", "clara smith", "clare maguire", "classified", "claude debussy", "claude king", "claudio arrau", "claudio monteverdi", "clawfinger", "clay aiken", "clay walker", "clean bandit", "cledus t. judd", "client", "cliff richard", "clifford t. ward", "climax blues band", "climie fisher", "clint black", "clipping.", "clique girlz", "close your eyes", "closet monster", "closterkeller", "cloud nothings", "clouds", "club 8", "club nouveau", "coal chamber", "cock robin", "cock sparrer", "cockney rejects", "coco montoya", "cocorosie", "cocteau twins", "code orange", "code red", "cody jinks", "cody simpson", "coffin break", "coil", "coko", "colbie caillat", "cold", "cold as life", "cold cave", "cold chisel", "cold world", "coldrain", "coldworker", "cole swindell", "colin blunstone", "colin hay", "colin james", "colin linden", "colin meloy", "collective soul", "collide", "collin raye", "colony 5", "color me badd", "colosseum", "colter wall", "combichrist", "comeback kid", "comes with the fall", "commander cody and his lost planet airmen", "commodores", "common", "common rider", "company flow", "company of thieves", "compton's most wanted", "con brio", "con funk shun", "con hunley", "conchita wurst", "conducting from the grave", "confederate railroad", "confide", "conjure one", "connie francis", "connie smith", "connie talbot", "conor oberst", "consequence", "converge", "conway twitty", "conway twitty & loretta lynn", "cool hand luke", "coolio", "cop shoot cop", "coptic rain", "corb lund", "corbin bleu", "corbin-hanner band", "corey cerovsek", "corey crowder", "corey hart", "corinne bailey rae", "corky and the juice pigs", "cornershop", "coronatus", "coroner", "corpus christi", "corrosion of conformity", "cory asbury", "cory branan", "cory morrow", "count bass d", "counterparts", "countess", "country joe and the fish", "country joe mcdonald", "courtney marie andrews", "coven 13", "covenant", "cowboy junkies", "cows", "cpr", "crack the sky", "cracker", "cradle of filth", "craig & dean", "craig cardiff", "craig david", "craig morgan", "craig's brother", "cranes", "crashdïet", "crazyeightyeight", "cream", "creature feature", "creed", "creedence clearwater revival", "creeper", "crest of darkness", "cretin", "crime & the city solution", "criminal", "crimson glory", "crimson moonlight", "crimson thorn", "crippled black phoenix", "crisis", "criss angel", "cromok", "crooked fingers", "crooked still", "crosby", "crosby", "crosby & nash", "cross canadian ragweed", "crossfaith", "crowded house", "crowder", "crown of thorns", "crown the empire", "cru", "cruachan", "crucial conflict", "crumbsuckers", "crush 40", "cryonic temple", "cryptic slaughter", "cryptic wintermoon", "cryptopsy", "crystal bernard", "crystal castles", "crystal fighters", "crystal gayle", "crystal kay", "crystal lewis", "crystal viper", "crystallion", "crywank", "cuban link", "cubanate", "cuby + blizzards", "cult of luna", "culture", "culture beat", "culture club", "cupcakke", "curl up and die", "curren$y", "current 93", "current swell", "cursed", "cursive", "curtis mayfield", "curtis stigers", "curved air", "cut copy", "cutting crew", "cyanotic", "cyferdyne", "cygnosic", "cymbals eat guitars", "cyne", "cynic", "cypecore", "cypress hill", "céline dion", "cœur de pirate", "d'espairsray", "d'sound", "d-a-d", "d.r.a.m.", "d.r.i.", "d12", "da vinci's notebook", "da' t.r.u.t.h.", "daan", "dads", "daforce", "daft punk", "dagoba", "damaged", "damh the bard", "damian marley", "damien dempsey", "damien jurado", "damien rice", "damon & naomi", "damon intrabartolo", "damone", "dan fogelberg", "dan hartman", "dan hill", "dan le sac vs scroobius pip", "dan seals", "dan tyminski", "dan zanes", "dana dirksen", "dana key", "dana winner", "danbert nobacon", "dance gavin dance", "danger danger", "danger doom", "danger radio", "dangerous toys", "dani siciliano", "daniel bedingfield", "daniel lanois", "daniel lavoie", "daniel m ller schott", "daniel o'donnell", "daniel shafran", "daniele liverani", "danielle peck", "danielson", "daniil trifonov", "danko jones", "danny brown", "danny gokey", "danny kaye", "danny kirwan", "danny michel", "danny wilde", "dante", "danzig", "dappled cities", "dar williams", "darden smith", "darin", "darius danesh", "darius rucker", "dark age", "dark angel", "dark fortress", "dark moor", "dark the suns", "dark tranquillity", "darkane", "darkest hour", "darkseed", "darkside", "darkthrone", "darkwell", "darlene zschech", "darlingside", "darren hanlon", "darren styles", "daryl hall", "darzamat", "das efx", "das pop", "dashboard confessional", "datarock", "daughter", "daughters", "dave alvin", "dave carter & tracy grammer", "dave clark five", "dave cousins", "dave davies", "dave dudley", "dave edmunds", "dave hollister", "dave rodgers", "dave stewart", "dave stewart & the spiritual cowboys", "dave van ronk", "davey suicide", "david & the citizens", "david and the giants", "david baerwald", "david ball", "david bazan", "david bromberg", "david byron", "david cassidy", "david coverdale", "david crosby", "david crowder band", "david essex", "david ford", "david geringas", "david gilmour", "david grisman", "david kersh", "david knopfler", "david lee murphy", "david lee roth", "david lindley", "david mallett", "david meece", "david oistrakh", "david rock feinstein", "david rovics", "david usher", "david wilcox", "dawes", "dawn landes", "dawn of ashes", "dawn richard", "dawnbringer", "day at the fair", "day of fire", "day26", "dayna kurtz", "days away", "daysend", "dc talk", "de la soul", "de lux", "de/vision", "de-phazz", "dead and divine", "dead by april", "dead can dance", "dead hand projekt", "dead infection", "dead kennedys", "dead meadow", "dead or alive", "dead prez", "dead sara", "dead stop", "dead to fall", "deadline", "deadlock", "deadmau5", "deadsoul tribe", "deadstar assembly", "dean brody", "dean martin", "deana carter", "dear", "dear and the headlights", "dear criminals", "dear reader", "death", "death angel", "death by stereo", "death cab for cutie", "death grips", "death in june", "death threat", "deathboy", "deathgaze", "deathspell omega", "deathstars", "deb talan", "debarge", "debbie gibson", "debbie harry", "debby boone", "deborah allen", "decapitated", "decoded feedback", "decrepit birth", "dee snider", "deep dish", "deep purple", "deepspace 5", "deer tick", "deerhunter", "def leppard", "defeater", "defiance", "defiance", "defleshed", "deftones", "degarmo and key", "deicide", "deine lakaien", "del shannon", "delain", "delays", "delbert mcclinton", "delerium", "delinquent habits", "delirious?", "deliverance", "delta goodrem", "delta moon", "delta spirit", "deltron 3030", "demi lovato", "demiricous", "demis roussos", "demolition hammer", "demon", "demon hunter", "demons & wizards", "den harrow", "deniece williams", "denis matsuev", "denison witmer", "dennis brown", "dennis deyoung", "dennis wilson", "denzel curry", "department of eagles", "derdian", "derek and the dominos", "derek minor", "derek webb", "descendents", "desert rose band", "desmond dekker", "despised icon", "destiny's child", "destroid", "destroy the runner", "destroyer", "destruction", "deströyer 666", "dethklok", "deuce", "devendra banhart", "devil doll", "devildriver", "devin townsend", "devin townsend project", "devlin", "devo", "devotchka", "devotchkas", "dew-scented", "dexter freebish", "dezarie", "dezperadoz", "diablo swing orchestra", "diabulus in musica", "diamond d", "diamond rio", "diana krall", "diana ross", "diana ross & the supremes and the temptations", "diana vickers", "diane cluck", "dianne reeves", "diary of dreams", "dick brave & the backbeats", "dick haymes", "dickey lee", "diddy", "dido", "die form", "die happy", "die krupps", "die sektor", "die so fluid", "die toten hosen", "die verbannten kinder evas", "die warzau", "die young", "dig", "digger", "dighayzoose", "digital summer", "diiv", "dikembe", "dilana", "dillinger four", "dillon", "dim mak", "dimmu borgir", "dinah washington", "dino ciani", "dinu lipatti", "dio", "dion", "dionne warwick", "dionysus", "diorama", "dir en grey", "dire straits", "dirty", "dirty heads", "dirty looks", "dirty pretty things", "disarmonia mundi", "disciple", "discipline", "disclosure", "discordance axis", "discount", "disfear", "dismantled", "dispatch", "disrupt", "disturbed", "diva destruction", "divinefire", "divinyls", "dixie chicks", "dizzee rascal", "dj antoine", "dj drama", "dj drama & lil wayne", "dj jazzy jeff & the fresh prince", "dj shadow", "dmitri shostakovich", "dmitry bashkirov", "dmx", "dnce", "do", "doc walker", "doc watson", "dock boggs", "dodgy", "dog eat dog", "dog fashion disco", "dogwood", "dokken", "dolly parton", "dolorian", "dolour", "dom pachino", "domenico scarlatti", "dominici", "domo genesis", "don francisco", "don gibson", "don henley", "don johnson big band", "don mclean", "don williams", "donald fagen", "donald lawrence", "donna fargo", "donna regina", "donna summer", "donnie mcclurkin", "donnie munro", "donovan", "doom", "dope", "dope stars inc.", "doris day", "doro", "dorothy", "dorsal atlântica", "dottie west", "double you", "doug macleod", "doug sahm", "dougie maclean", "douwe bob", "dove cameron", "dover", "doves", "down", "down by law", "down with webster", "downplay", "doyle bramhall", "doyle bramhall ii", "doyle lawson & quicksilver", "dr. acula", "dr. alban", "dr. dog", "dr. dre", "dr. feelgood", "dr. hook & the medicine show", "dr. john", "dr. sin", "draconian", "drag the river", "dragonette", "dragonforce", "dragonheart", "dragonland", "drain sth", "drake", "drake bell", "drapht", "dreadful shadows", "dream evil", "dream street", "dream theater", "dreamland", "dreams of sanity", "dreamtale", "dreezy", "drenge", "dressy bessy", "drive", "drive-by truckers", "drop dead", "dropdead", "dropkick murphys", "drowning pool", "drowning the light", "dual core", "dubioza kolektiv", "dubstar", "duff mckagan", "duff mckagan's loaded", "duffy", "duke ellington", "dum dum girls", "dune", "dust of basement", "dustin kensrue", "dusty springfield", "dutch rebelle", "dwele", "dwight twilley", "dwight yoakam", "dying fetus", "dynazty", "dyscarnate", "dystopia", "dälek", "dååth", "dødheimsgard", "e", "e-40", "e-rotic", "e-type", "e.g. daily", "e.s.g.", "eagles", "eagles of death metal", "eamon", "earl scruggs", "earl sweatshirt", "earl thomas conley", "earl wild", "earth", "earth and fire", "earth crisis", "eartha", "eartha kitt", "east west", "easton corbin", "easy rider", "easy star all-stars", "easyworld", "eazy-e", "echolyn", "echosmith", "eclipse", "ed bruce", "ed harcourt", "ed schrader's music beat", "ed sheeran", "eddi reader", "eddie cochran", "eddie degarmo", "eddie floyd", "eddie from ohio", "eddie money", "eddie murphy", "eddie rabbitt", "eddie vedder", "eddy arnold", "eddy grant", "eden synthetic corps", "eden's curse", "edenbridge", "edgar broughton band", "edgar winter", "edge of dawn", "edge of sanity", "edguy", "edison glass", "editors", "edl", "edvard grieg", "edward elgar", "edward maya", "edwin", "edwin fischer", "edyta górniak", "egypt central", "eiffel 65", "eighteen visions", "eilera", "einherjer", "einstürzende neubauten", "eisley", "ektomorf", "el debarge", "el perro del mar", "el-p", "elane", "elbow", "eleanor friedberger", "electrelane", "electric guest", "electric light orchestra", "electric president", "electric six", "electric valentine", "electric wizard", "elefant", "elegant machinery", "elegy", "element 101", "elend", "eleni mandell", "elephant man", "elevation worship", "eleventyseven", "elf power", "eli sostre", "eliane elias", "elijah blake", "elis", "elisa", "elitist", "eliza neals", "elizabeth cook", "elizabeth shepherd", "ella fitzgerald", "ella fitzgerald & louis armstrong", "ella mae morse", "ella mai", "elle king", "elle milano", "ellie goulding", "elliott brood", "elliott smith", "elliott yamin", "elliphant", "ellis paul", "elly ney", "elmore james", "elo part ii", "eloy", "elton john", "eluveitie", "elvenking", "elvin bishop", "elvis costello", "elvis costello & the attractions", "elvis depressedly", "elvis presley", "elysian fields", "elysium", "emanuel", "emanuel feuermann", "emeli sandé", "emerald", "emerson", "emerson drive", "emery", "emf", "emigrate", "emika", "emil gilels", "emilie autumn", "emily haines", "emilíana torrini", "emin", "eminem", "emitt rhodes", "emma bunton", "emma ruth rundle", "emmerson nogueira", "emmure", "emmy rossum", "emmy the great", "emmylou harris", "emperor", "empire of the sun", "empire! empire! (i was a lonely estate)", "empress of", "empyrium", "en vogue", "enchant", "end of you", "endanger", "endo", "england dan & john ford coley", "enigma", "enochian crescent", "enon", "enrico caruso", "enrique iglesias", "ensiferum", "ensign", "enslaved", "enter the haggis", "entombed", "enuff z'nuff", "envy", "envy on the coast", "epic rap battles of history", "epica", "epmd", "equatronic", "erasure", "eric andersen", "eric b. & rakim", "eric bibb", "eric bogle", "eric burdon", "eric burdon & the animals", "eric burdon & war", "eric carmen", "eric church", "eric clapton", "eric fish", "eric johnson", "eric lindell", "eric martin", "eric saade", "eric stewart", "eric woolfson", "eric's trip", "erick sermon", "erin mccarley", "ernest tubb", "ernest tubb and loretta lynn", "ernestine anderson", "erra", "erykah badu", "esben and the witch", "escape the fate", "esham", "eskimo callboy", "esoteric", "esperanza spalding", "esther ofarim", "esther phillips", "eternal", "eternal tears of sorrow", "ethel merman", "ethel waters", "etta james", "eug ne ysa e", "eugene mcguinness", "eugenio finardi", "europe", "eurythmics", "eva cassidy", "evan taubenfeld", "evanescence", "evergreen terrace", "evergrey", "everlife", "evermore", "every time i die", "everyone everywhere", "everything but the girl", "evgeny kissin", "evidence", "evils toy", "evocation", "evoken", "ewan maccoll", "ewert and the two dragons", "example", "excision", "exciter", "exile", "exo", "exodus", "exposé", "extreme", "extreme noise terror", "eydie gorme", "eyedea & abilities", "eyes set to kill", "ezio", "ezra furman", "f-minus", "f.k.ü.", "fabrizio faniello", "face to face", "faces", "facing new york", "faderhead", "fair to midland", "fair warning", "fairground attraction", "fairport convention", "fairyland", "faith assembly", "faith evans", "faith hill", "faith no more", "faithless", "fake?", "falco", "falconer", "falconshield", "falkenbach", "fall of the leafe", "fall out boy", "falling in reverse", "falling up", "fallujah", "family force 5", "fancy", "fantasia", "far east movement", "far-less", "faron young", "fastball", "faster pussycat", "fat joe", "fatboy slim", "fates warning", "father", "fatherson", "fats domino", "fats waller", "fatso jetson", "faun fables", "faunts", "fay lovsky", "faz l say", "fear", "fear", "fear factory", "fear of domination", "feargal sharkey", "feed her to the sharks", "feeling left out", "feist", "felix mendelssohn bartholdy", "fenix tx", "fergie", "ferlin husky", "fernando ortega", "ferraby lionheart", "ferruccio busoni", "fetty wap", "fever ray", "fever tree", "ffh", "fgfc820", "fiction family", "fiddler's green", "field mob", "field music", "fields of the nephilim", "fifteen", "fifth angel", "fifth harmony", "fight amp", "fightstar", "figure four", "film school", "finch", "fiona", "fiona apple", "fiona boyes", "fionn regan", "firefall", "fireflight", "firehouse", "firewind", "first blood", "fischer-z", "fischerspooner", "fish", "fishboy", "fisher", "fit for a king", "fit for an autopsy", "five", "five finger death punch", "five for fighting", "five iron frenzy", "flame", "flashlight brown", "flatfoot 56", "flatsound", "flatt & scruggs", "flaw", "fleet foxes", "fleetwood mac", "flesh field", "flesh-n-bone", "fleurie", "flight of the conchords", "flipsyde", "flo rida", "flobots", "flora cash", "florence + the machine", "florida georgia line", "florrie", "flowing tears", "flunk", "flying lotus", "flyleaf", "fm", "fm laeti", "fm static", "foals", "foetus", "fog", "foghat", "folly & the hunter", "fool's garden", "for all those sleeping", "for the fallen dreams", "for today", "forbidden", "force majeure", "foreigner", "forever changed", "forever slave", "forever the sickest kids", "forevermore", "forgive durden", "forgotten tales", "fort minor", "four letter lie", "four tops", "fourplay", "foxes", "foxy brown", "fozzy", "fr d ric chopin", "fractured", "fraggle rock", "fran ois couperin", "francis dunnery", "francis poulenc", "franco battiato", "frank black", "frank hutchison", "frank ocean", "frank proffitt", "frank sinatra", "frank stokes", "frank turner", "frank zappa", "frankenstein drag queens from planet 13", "frankie goes to hollywood", "frankie j", "frankie lee sims", "frankie lymon & the teenagers", "frankie valli", "frankjavcee", "franz ferdinand", "franz liszt", "franz schubert", "françoise hardy", "freak kitchen", "freakwater", "fred astaire", "fred eaglesmith", "fred neil", "fred penner", "freddie hart", "freddie jackson", "freddie king", "freddie mercury", "freddy fender", "fredrika stahl", "free", "free dominguez", "free throw", "freedom call", "freedom fry", "freelance whales", "freesscape", "freeway", "freezepop", "french kicks", "french montana", "frenzal rhomb", "freshlyground", "freya", "frida", "frida hyvönen", "friend 'n fellow", "frightened rabbit", "fritz kalkbrenner", "fritz kreisler", "froggy fresh", "from ashes to new", "from autumn to ashes", "from first to last", "from good homes", "front 242", "front line assembly", "frost", "frozen ghost", "frozen plasma", "fruit bats", "fruupp", "fu manchu", "fuck", "fucked up", "fugazi", "fugees", "fujiya & miyagi", "full blown chaos", "fun boy three", "fun lovin' criminals", "fun people", "fun.", "funeral", "funeral for a friend", "funkadelic", "funker vogt", "furry lewis", "further seems forever", "fury", "fury in the slaughterhouse", "future", "future islands", "fyfe", "g herbo", "g-unit", "g.g.f.h.", "g4", "gabriel brown", "gabriel cyphre", "gabriel faur ", "gabriel kahane", "gabriella cilmi", "gabrielle", "gabrielle aplin", "gaelic storm", "gaetano donizetti", "gaia epicus", "gaither vocal band", "galactic cowboys", "galahad", "galantis", "galaxie 500", "gallows", "galt macdermot", "game theory", "gamma ray", "gandalf's fist", "gang starr", "ganggajang", "gangsta boo", "gare du nord", "gareth gates", "garfunkel and oates", "garnet rogers", "garrick ohlsson", "garrison starr", "garth brooks", "gary allan", "gary barlow", "gary brooker", "gary chapman", "gary clark jr.", "gary glitter", "gary jules", "gary lewis & the playboys", "gary moore", "gary morris", "gary numan", "gary puckett & the union gap", "gary stewart", "gary u.s. bonds", "gary wright", "gatsbys american dream", "gautier capu on", "gazebo", "gazpacho", "gehenna", "gene autry", "gene clark", "gene pitney", "gene simmons", "gene vincent", "gene watson", "general public", "generation x", "genesis", "genghis tron", "genitorturers", "gentle giant", "gentleman", "geoff berner", "geoff farina", "geoff moore", "georg philipp telemann", "george", "george benson", "george clinton", "george enescu", "george ezra", "george formby", "george frideric handel", "george gershwin", "george hamilton iv", "george harrison", "george jones", "george michael", "george morgan", "george nozuka", "george strait", "george thorogood & the destroyers", "georges bizet", "georges cziffra", "georgie fame", "gerald levert", "gerald moore", "gerry and the pacemakers", "gerry rafferty", "geto boys", "ghost", "ghost brigade", "ghost dance", "ghost ship", "ghost town", "ghostface killah", "ghostlimb", "ghostpoet", "ghoti hook", "ghoul", "giacomo puccini", "giant sand", "giant squid", "gideon", "gidon kremer", "gigi d'agostino", "gil scott-heron", "gil shaham", "gilbert and sullivan", "gilbert o'sullivan", "gilberto gil", "gilby clarke", "giles", "giles and fripp", "ginger", "gino vannelli", "ginuwine", "gioacchino rossini", "giorgio moroder", "giovanni battista pergolesi", "girl talk", "girlicious", "girlpool", "girls", "girls aloud", "girls in hawaii", "girls rituals", "girls under glass", "girls' generation", "girlschool", "giuseppe tartini", "giuseppe verdi", "gladys knight", "gladys knight & the pips", "glass hammer", "glass harp", "glass tiger", "glasseater", "glee cast", "glen campbell", "glenn frey", "glenn gould", "glenn hughes", "glenn kaiser", "glenn kaiser band", "glenn medeiros", "glenn miller", "glenn tipton", "gloria estefan", "gloria gaynor", "glorior belli", "gluecifer", "go periscope", "go west", "goat of mendes", "god forbid", "god lives underwater", "god module", "godhead", "godley & creme", "gods paparazzi", "godsmack", "gojira", "golden earring", "golden gate quartet", "golden smog", "goldfinger", "goldfrapp", "gong", "good clean fun", "good old war", "good rats", "good riddance", "good shoes", "goodie mob", "gordon bok", "gordon downie", "gordon haskell", "gordon lightfoot", "gorefest", "goretrade", "gorgeous", "gorguts", "gorillaz", "gorky's zygotic mynci", "gorod", "gossip", "gothminister", "gotthard", "gov't mule", "government issue", "gowan", "grace jones", "grace vanderwaal", "grade", "graf orlock", "graham central station", "graham colton", "graham nash", "gram parsons", "gramatik", "grammatrain", "grand funk railroad", "grand puba", "grandaddy", "grandmaster flash", "grandpa jones", "grant hart", "grant lee buffalo", "grant-lee phillips", "grass widow", "grateful dead", "grave", "grave digger", "grave maker", "gravediggaz", "graveland", "graveworm", "graveyard", "grayson & whitter", "graziano romani", "great big sea", "great captain!", "great lake swimmers", "great northern", "great white", "greeley estates", "green carnation", "green jellÿ", "green river ordinance", "greensky bluegrass", "greenslade", "greg brown", "greg lake", "greg long", "gregg allman", "gregor piatigorsky", "gregorian", "gregory alan isakov", "gregory and the hawk", "gregory isaacs", "gregory porter", "grendel", "gretchen wilson", "grey delisle", "grief", "grieves", "grigory sokolov", "grimes", "grimskunk", "grinderman", "grinspoon", "grits", "grizzly bear", "grobschnitt", "groundation", "grouplove", "guano apes", "guardian", "guerilla maab", "guerilla toss", "guillemots", "guiomar novaes", "gun", "gungor", "guns n' roses", "guru", "gusgus", "gustav holst", "gustav mahler", "guy", "guy clark", "guy mitchell", "guy sebastian", "guy verlinde", "gwar", "gwen stacy", "gwen stefani", "gym class heroes", "gza genius", "gzr", "h-blockx", "h-town", "h.e.a.t", "h.e.r.", "h.p. lovecraft historical society", "hackneyed", "haddaway", "hadise", "hadouken!", "haemorrhage", "haerts", "hafdís huld", "hagalaz' runedance", "haggard", "hail of bullets", "haim", "haircut 100", "haken", "halestorm", "haley reinhart", "half man half biscuit", "half-a-mill", "halford", "halifax", "hall & oates", "hallelujah the hills", "halou", "halsey", "hammerfall", "hammock", "hands", "handsome ghost", "hangar", "hangnail", "hania", "hank cochran", "hank green", "hank locklin", "hank snow", "hank thompson", "hank williams", "hank williams", "hank williams iii", "hannah fury", "hanne hukkelberg", "hanne kah", "hanoi rocks", "hans theessink", "hanson", "hanzel und gretyl", "hap palmer", "happy days", "happy monster band", "har mar superstar", "hard-fi", "hardcore superstar", "hardline", "harem scarem", "harlan howard", "harley poe", "harold melvin & the blue notes", "harry and the potters", "harry belafonte", "harry chapin", "harry connick", "harry james", "harry manx", "harry nilsson", "haste the day", "hate", "hate dept.", "hate eternal", "hatebreed", "hateen", "hatesphere", "haujobb", "have a nice life", "havoc", "hawk nelson", "hawksley workman", "hawkwind", "hawthorne heights", "hayden", "hayes carll", "hayley kiyoko", "hayley westenra", "hayseed dixie", "haystak", "hazel dickens", "hazel o'connor", "hb", "he is legend", "he is we", "head automatica", "head east", "headlights", "hear'say", "heart", "heart in hand", "heart of a coward", "heartless bastards", "heartsounds", "heartsrevolution", "heathen", "heather alexander", "heather dale", "heather headley", "heather nova", "heaven & earth", "heaven & hell", "heaven 17", "heaven shall burn", "heavens edge", "heavens to betsy", "heavy heavy low low", "hector berlioz", "heffron drive", "heideroosjes", "heinrich schiff", "heitor villa lobos", "helalyn flowers", "helen baylor", "helen humes", "helen kane", "helen o'connell", "helix", "hell is for heroes", "hell or highwater", "hell razah", "hellhammer", "hello saferide", "hellogoodbye", "helloween", "hellsongs", "hellyeah", "helmet", "helstar", "hem", "henry fiat's open sore", "henry jamison", "henry krieger", "henry mancini", "henry purcell", "henryk szeryng", "henson cargill", "her space holiday", "herb alpert & the tijuana brass", "herbert grönemeyer", "herbie hancock", "hercules & love affair", "here come the mummies", "herman brood", "herman's hermits", "hexrx", "hey", "hey mercedes", "hey monday", "hey ocean!", "hey rosetta!", "hezekiah walker", "hi-tek", "hiatus kaiyote", "hidden in plain view", "hieroglyphics", "high and mighty color", "high inergy", "high on fire", "high places", "high school football heroes", "highasakite", "highway 101", "hilary duff", "hilary hahn", "hillsong", "hilltop hoods", "hinder", "hinds", "hirax", "his hero is gone", "his name is alive", "his statue falls", "hoagy carmichael", "hocico", "hodgy beats", "hollow haze", "holly cole", "holly dunn", "holly golightly", "holly miranda", "holly throsby", "holy ghost!", "holy soldier", "homer & jethro", "honeymoon suite", "honne", "hoobastank", "hoodoo gurus", "hoods", "hop along", "hopesfall", "horrified", "horrorpops", "horse feathers", "horse the band", "hot boy$", "hot chelle rae", "hot chocolate", "hot dad", "hot rod circuit", "hot snakes", "hot tuna", "hot water music", "hound dog taylor", "houndmouth", "hourglass", "house of lords", "house of pain", "house vs. hurricane", "housefires", "how to dress well", "howard jones", "howard shore", "howe gelb", "howie day", "howlin rain", "howlin' wolf", "howling bells", "hoyt axton", "hozier", "hubert kah", "hue and cry", "hugh laurie", "hughes turner project", "hum", "hundreds", "hundredth", "hungry lights", "hungry lucy", "hunters & collectors", "huntingtons", "hurrah!", "hurricane", "hurt", "hurts", "huski", "husky rescue", "hussein fatal", "hybrid", "hypocrisy", "i", "i am abomination", "i am ghost", "i blame coco", "i can make a mess like nobody's business", "i declare war", "i fight dragons", "i killed the prom queen", "i monster", "i see stars", "i set my friends on fire", "i'm from barcelona", "i.d.o.4.", "iamamiwhoami", "iamx", "ian & sylvia", "ian anderson", "ian dury and the blockheads", "ian gillan", "ian hunter", "ian moore", "ian thomas", "ibeyi", "ice ages", "ice cube", "ice mc", "ice nine kills", "ice-t", "iceage", "iced earth", "icehouse", "icon for hire", "icon of coil", "icona pop", "icons of filth", "ida", "ida cox", "ida haendel", "idiot stare", "idle cure", "iggy azalea", "iggy pop", "ignacy jan paderewski", "ignaz friedman", "ignite", "igor stravinsky", "ihsahn", "iio", "ike & tina turner", "ikon", "ill bill", "ill niño", "illogic", "ilse delange", "ima robot", "imagination movers", "imelda may", "immaculate fools", "immaculate machine", "immortal", "immortal technique", "imogen heap", "impaled nazarene", "impending doom", "imperative reaction", "imperia", "in dying arms", "in fear and faith", "in flames", "in hearts wake", "in strict confidence", "in the midst of lions", "in the woods...", "in this moment", "incantation", "incubus", "indecision", "india.arie", "indica", "indigo girls", "infected mushroom", "infected rain", "infectious grooves", "inferi", "informatik", "information society", "ingested", "inhale exhale", "inna", "inquisition", "insane clown posse", "insomnium", "inspectah deck", "instalok", "intense", "interface", "into eternity", "into it. over it.", "intwine", "ion dissonance", "iona", "iona brown", "iq", "iration", "irene cara", "iris dement", "irma thomas", "iron & wine", "iron butterfly", "iron fire", "iron maiden", "iron reagan", "ironsword", "irving", "is me", "isaac alb niz", "isaac hayes", "isac elliot", "isgaard", "isis", "islands", "isley jasper isley", "israel houghton", "israel kamakawiwo'ole", "israel vibration", "issues", "it breathes", "it dies today", "it lives", "it prevails", "itzhak perlman", "ivan moravec", "ivoryline", "ivy sole", "iwrestledabearonce", "iyaz", "izegrim", "izz", "izzy stradlin", "j church", "j dilla", "j moss", "j-zone", "j. cole", "j. tillman", "j.b. lenoir", "j.d. souther", "j.j. cale", "ja rule", "jaap schr der", "jack blanchard & misty morgan", "jack bruce", "jack greene", "jack howard", "jack ingram", "jack johnson", "jack jones", "jack savoretti", "jack the lad", "jack white", "jackie deshannon", "jackie evancho", "jackie gleason", "jackie lomax", "jackie wilson", "jackson browne", "jackson c. frank", "jackson heights", "jacob banks", "jacob's dream", "jacqueline du pr ", "jacques offenbach", "jade 4u", "jade valerie", "jade warrior", "jaden smith", "jag panzer", "jagged edge", "jaill", "jake owen", "jakob dylan", "jamala", "james blake", "james blunt", "james brown", "james cotton", "james gang", "james hunter", "james ingram", "james keelaghan", "james labrie", "james marsters", "james mcmurtry", "james morrison", "james otto", "james taylor", "james young", "jamestown story", "jamey johnson", "jamie cullum", "jamie grace", "jamie lawson", "jamie lidell", "jamie o'neal", "jamie winchester", "jamie's elsewhere", "jan & dean", "jan howard", "jan wayne", "jan werner", "jana mashonee", "jandek", "jane child", "jane monheit", "janelle monáe", "janet jackson", "janie fricke", "janis ian", "janis joplin", "jann arden", "japan", "japandroids", "jarboe", "jars of clay", "jascha heifetz", "jasmine thompson", "jasmine v", "jason aldean", "jason anderson", "jason collett", "jason derulo", "jason donovan", "jason falkner", "jason isbell", "jason mraz", "jason webley", "javier", "javier mendoza", "jay & the americans", "jay farrar", "jay ferguson", "jay reatard", "jay rock", "jay sean", "jay-jay johanson", "jay-z", "jaya the cat", "jaymay", "jazmine sullivan", "jbm", "jc chasez", "jean baptiste lully", "jean beauvoir", "jean michel jarre", "jean philippe rameau", "jean shepard", "jean sibelius", "jeanette biedermann", "jeannie c. riley", "jeannie seely", "jedi mind tricks", "jeff beck", "jeff deyo", "jeff healey", "jeff lynne", "jeff scott soto", "jeff the brotherhood", "jeff tweedy", "jeff wayne", "jeff williams", "jefferson airplane", "jefferson starship", "jeffree star", "jeffrey lewis", "jello biafra", "jellyfish", "jeniferever", "jennifer kimball", "jennifer lopez", "jennifer love hewitt", "jennifer nettles", "jennifer rush", "jennifer warnes", "jenny hval", "jenny lewis", "jenny o.", "jens lekman", "jeremih", "jeremy camp", "jeremy enigk", "jeremy larson", "jeremy messersmith", "jeremy riddle", "jeremy spencer", "jermaine jackson", "jermaine stewart", "jerry butler", "jerry cantrell", "jerry garcia", "jerry garcia band", "jerry jeff walker", "jerry lee lewis", "jerusalem", "jess glynne", "jess moskaluke", "jesse harris", "jesse harris & the ferdinandos", "jesse malin", "jesse mccartney", "jessi colter", "jessica andrews", "jessica harp", "jessica lea mayfield", "jessica pratt", "jessica simpson", "jessie j", "jessie james", "jessie ware", "jesus culture", "jesus on extasy", "jet life", "jet set satellite", "jethro tull", "jets overhead", "jett rebel", "jewel", "jhené aiko", "jigsaw", "jill barber", "jill scott", "jill sobule", "jill tracy", "jillette johnson", "jillian aversa", "jim brickman", "jim capaldi", "jim croce", "jim cuddy", "jim ed brown", "jim guthrie", "jim jackson", "jim james", "jim jones", "jim kweskin", "jim lauderdale", "jim messina", "jim noir", "jim reeves", "jim's big ego", "jimi hendrix", "jimi jamison", "jimmie rodgers", "jimmie vaughan", "jimmy barnes", "jimmy buffett", "jimmy cliff", "jimmy dawkins", "jimmy dean", "jimmy dorsey", "jimmy nail", "jimmy reed", "jimmy rushing", "jimmy somerville", "jimmy webb", "jimmy witherspoon", "jin akanishi", "jinjer", "jinkx monsoon", "jj", "jj grey & mofro", "jj72", "jls", "jme", "jmsn", "jo stafford", "joan armatrading", "joan baez", "joan jett and the blackhearts", "joan of arc", "joan osborne", "joanie sommers", "joanna newsom", "job for a cowboy", "jocelyn & chris arndt", "jocelyn enriquez", "jodeci", "jody miller", "jody watley", "joe", "joe bonamassa", "joe budden", "joe cocker", "joe diffie", "joe ely", "joe henry", "joe hill", "joe jackson", "joe mcelderry", "joe nichols", "joe purdy", "joe walsh", "joe williams", "joel plaskett", "joel plaskett emergency", "joey + rory", "joey mcintyre", "joey tempest", "johan", "johann christian bach", "johann pachelbel", "johann sebastian bach", "johann strauss ii", "johannes brahms", "johannes moser", "john anderson", "john barrowman", "john brown's body", "john browning", "john cena", "john cooper clarke", "john d. loudermilk", "john denver", "john denver & the muppets", "john doe", "john eddie", "john elefante", "john entwistle", "john farnham", "john fogerty", "john foxx", "john hammond", "john hiatt", "john illsley", "john k. samson", "john kay", "john lee hooker", "john lee hooker and canned heat", "john legend", "john lennon", "john lennon & yoko ono", "john martyn", "john maus", "john mayall", "john mayer", "john mccormack", "john mccutcheon", "john mellencamp", "john michael montgomery", "john michael talbot", "john miles", "john norum", "john oates", "john ogdon", "john p. kee", "john parr", "john pizzarelli", "john popper", "john prine", "john ralston", "john reuben", "john schneider", "john sebastian", "john stewart", "john waite", "john wesley", "john wesley harding", "john west", "john wetton", "johnnie allan", "johnny burnette", "johnny cash", "johnny cash & june carter cash", "johnny clegg", "johnny clegg & savuka", "johnny crash", "johnny duncan", "johnny gill", "johnny hallyday", "johnny hates jazz", "johnny horton", "johnny kidd & the pirates", "johnny mathis", "johnny mercer", "johnny paycheck", "johnny reid", "johnny rivers", "johnny rodriguez", "johnny shines", "johnny thunders", "johnny winter", "johnossi", "joji", "jojo", "jolie holland", "jon allen", "jon and vangelis", "jon anderson", "jon bon jovi", "jon foreman", "jon mclaughlin", "jon oliva's pain", "jon randall", "jon secada", "jonah matranga", "jonas brothers", "jonatha brooke", "jonathan larson", "jonathan richman and the modern lovers", "jonathan thulin", "joni mitchell", "jonna lee", "jonny diaz", "jonny lang", "jools holland", "jordan pruitt", "jordan smith", "jordin sparks", "jorge bolet", "jorma kaukonen", "jorn", "josef lh vinne", "josef suk", "joseph arthur and the lonely astronauts", "joseph haydn", "joseph szigeti", "josephine foster", "josh garrels", "josh groban", "josh ritter", "josh rouse", "josh turner", "josh white", "josh woodward", "joshua bell", "joshua kadison", "joshua perahia", "joss stone", "josé feliciano", "josé gonzález", "journey", "joy", "joy division", "joy electric", "joy williams", "joy zipper", "jp cooper", "jr.", "jr.", "jr.", "json", "jt music", "jt the bigga figga", "jucifer", "judas priest", "judie tzuke", "judy collins", "judy garland", "juelz santana", "juice", "juice newton", "juice wrld", "juicy j", "jukebox the ghost", "julee cruise", "julia holter", "julia jacklin", "julia stone", "julian lennon", "julian lloyd webber", "julie andrews", "julie doiron", "julie miller", "julie roberts", "julien-k", "juliette and the licks", "julio iglesias", "juluka", "jump", "jump5", "june carter cash", "june christy", "june of 44", "jungle rot", "junior wells", "junius", "junkie xl", "jupiter apple", "jupiter one", "jurassic 5", "just jinger", "just surrender", "justin bieber", "justin hayward", "justin nozuka", "justin timberlake", "justin townes earle", "justin young", "k'jon", "k'naan", "k-ci & jojo", "k. michelle", "k.d. lang", "k.flay", "k.t. oslin", "kacey musgraves", "kahimi karie", "kaipa", "kaiser chiefs", "kaiser mansfield", "kajagoogoo", "kalan porter", "kaledon", "kali uchis", "kalmah", "kamelot", "kandi", "kane & abel", "kansas", "kanye west", "karate", "karate high school", "kardinal offishall", "karen clark sheard", "karen elson", "karen matheson", "kari jobe", "kari peitsamo", "karin park", "karine polwart", "karl wolf", "karla bonoff", "karmakanic", "karnivool", "karyn white", "kasey chambers", "kaskade", "kat deluna", "kat-tun", "kataklysm", "katatonia", "kate & anna mcgarrigle", "kate alexa", "kate miller-heidke", "kate nash", "kate rusby", "kate ryan", "kate voegele", "katharine mcphee", "katherine jenkins", "kathleen edwards", "kathryn scott", "kathryn williams", "katie armiger", "katie melua", "katy perry", "katy rose", "kay starr", "kayo dot", "kaysha", "kb", "keaton henson", "keb' mo'", "keely smith", "kehlani", "keith & kristyn getty", "keith green", "keith moon", "keith murray", "keith richards", "keith sweat", "keith urban", "keith whitley", "kekal", "keke palmer", "keke wyatt", "keldian", "kelela", "kelis", "kelley stoltz", "kelli ali", "kellie pickler", "kelly clarkson", "kelly osbourne", "kelly price", "kelly willis", "ken hensley", "ken stringfellow", "kendrick lamar", "kenny chesney", "kenny g", "kenny loggins", "kenny rogers", "kenny wayne shepherd", "kenny white", "kent", "keren ann", "keri hilson", "kero kero bonito", "kerrs pink", "kerry livgren", "keshia chanté", "kevin fowler", "kevin gilbert", "kevin lyttle", "kevin rudolf", "kevin welch", "kevorkian death cycle", "keziah jones", "khalid", "khia", "khoma", "kick axe", "kid cudi", "kid down", "kid rock", "kidneythieves", "kidz bop", "kierra sheard", "kiethevez", "kill switch...klick", "kill your idols", "killa kyleon", "killer dwarfs", "killing joke", "killswitch engage", "killwhitneydead", "kim boyce", "kim carnes", "kim churchill", "kim mitchell", "kim wilde", "kimberley locke", "kimbra", "king adora", "king charles", "king creosote", "king crimson", "king diamond", "king gizzard & the lizard wizard", "king kobra", "king missile", "king tee", "king's x", "kingcrow", "kingdom come", "kingdom of sorrow", "kings of convenience", "kingston wall", "kip winger", "kira isabella", "kirk franklin", "kishi bashi", "kiss", "kite", "kittie", "kitty wells", "kiuas", "kix", "kj-52", "klaatu", "klone", "kmfdm", "knife party", "knightowl", "knights of the abyss", "knut", "kobra and the lotus", "koda kumi", "kodak black", "koffin kats", "koko taylor", "kokomo arnold", "kompressor", "kontrust", "kool & the gang", "kool keith", "koritni", "korn", "korpiklaani", "kosheen", "kotipelto", "kotoko", "kovacs", "kraan", "kraftwerk", "krayzie bone", "kreator", "krewella", "krezip", "kris kristofferson", "krisiun", "krisma", "kristene dimarco", "kristian stanfill", "kristin hersh", "kristine w", "kristinia debarge", "kristy lee cook", "kristy thirsk", "krizz kaliko", "kronos", "krs-one", "krypteria", "krystal meyers", "krystian zimerman", "kt tunstall", "kula shaker", "kultur shock", "kungfu rick", "kunt and the gang", "kurt elling", "kurt nilsen", "kurt vile", "kurupt", "kutt calhoun", "kygo", "kylesa", "kylie minogue", "kyung wha chung", "kyuss", "l'âme immortelle", "l.a. guns", "l.t.d.", "la bionda", "la bouche", "la coka nostra", "la dispute", "la roux", "la sera", "la the darkman", "la toya jackson", "labyrinth", "lacrimas profundere", "lacrimosa", "lacrosse", "lacuna coil", "lacy j. dalton", "lady antebellum", "lady gaga", "lady sovereign", "ladyhawke", "ladysmith black mambazo", "ladytron", "lagwagon", "laibach", "laika", "lake", "lake & palmer", "lake of tears", "lake street dive", "lalah hathaway", "lalaine", "laleh", "lali puna", "lamb", "lamb of god", "lambchop", "lamont dozier", "lana del rey", "lana lane", "land of talk", "landmine marathon", "landon pigg", "lang lang", "lani hall", "lany", "lara fabian", "lard", "lari white", "larry gatlin & the gatlin brothers", "larry norman", "larry santos", "larry sparks", "larry the cable guy", "larue", "lasgo", "last train home", "last tuesday", "late tuesday", "laura branigan", "laura cantrell", "laura fygi", "laura gibson", "laura marling", "laura pausini", "lauren daigle", "lauren hoffman", "laurie anderson", "lauryn hill", "lawnmower deth", "lay low", "layzie bone", "lazar berman", "lcd soundsystem", "le butcherettes", "le tigre", "lea michele", "lea salonga", "lead belly", "leatherface", "leaves' eyes", "lebanon hanover", "lecrae", "led zeppelin", "lee aaron", "lee brice", "lee dorsey", "lee fields", "lee greenwood", "lee hazlewood", "lee kernaghan", "lee ryan", "leeland", "left alone", "left spine down", "lefty frizzell", "leftöver crack", "legendary shack shakers", "legion of the damned", "leif ove andsnes", "leigh nash", "lemar", "lemon jelly", "lemuria", "lena", "lena horne", "lena katina", "lene lovich", "lenka", "lenny kravitz", "leo jan ek", "leo sayer", "leon fleisher", "leon redbone", "leon russell", "leona lewis", "leonard bernstein", "leonard cohen", "leonard nimoy", "leonard pennario", "leonardo's bride", "leonid kogan", "leopold godowsky", "leprous", "leroy anderson", "leroy carr", "leroy hutson", "les claypool", "les fleur de lys", "les humphries singers", "les savy fav", "lesley gore", "leslie hall", "leslie west", "less than jake", "lethian dreams", "letlive", "level", "level 42", "leviathan", "levon helm", "leæther strip", "liam finn", "liars", "libera", "liberty x", "liege lord", "lifetime", "lighthouse family", "lightnin' hopkins", "lightning seeds", "lights", "lights of euphoria", "like moths to flames", "lil b", "lil baby", "lil boosie", "lil johnson", "lil jon", "lil jon & the east side boyz", "lil mama", "lil pump", "lil skies", "lil son jackson", "lil suzy", "lil wayne", "lil wyte", "lil yachty", "lil' flip", "lil' keke", "lila mccann", "lily & madeleine", "lily allen", "lilys", "limahl", "limbeck", "limp", "limp bizkit", "lin-manuel miranda", "linda davis", "linda perry", "linda ronstadt", "lindi ortega", "lindisfarne", "lindsay lohan", "linkin park", "linton kwesi johnson", "linus of hollywood", "lionel hampton", "lionel richie", "lisa brokop", "lisa ekdahl", "lisa hannigan", "lisa marie presley", "lisa thiel", "listener", "lit", "lita ford", "little big", "little big town", "little boots", "little children", "little dragon", "little jimmy dickens", "little man tate", "little milton", "little mix", "little richard", "little river band", "little simz", "little walter", "little willie john", "liv kristine", "living colour", "living legends", "living sacrifice", "livingston taylor", "liz durrett", "liza anne", "lizz wright", "lizzy borden", "ll cool j", "lmfao", "lobo", "local natives", "lock up", "locksley", "lodger", "loggins & messina", "logic", "lola monroe", "lolo", "london elektricity", "london grammar", "london symphony", "londonbeat", "lonely kings", "lonestar", "loney", "long john baldry", "longfellow", "longwave", "lonnie donegan", "lonnie johnson", "look what i did", "lord belial", "lord of the lost", "lorde", "lordi", "lords of acid", "lords of black", "loreena mckennitt", "loretta lynn", "lorna shore", "lorrie morgan", "los campesinos!", "los lobos", "los pericos", "lost in tears", "lotte kestner", "lou barlow", "lou bega", "lou christie", "lou gramm", "lou rawls", "lou reed", "lou rhodes", "loudon wainwright iii", "louis armstrong", "louis jordan", "louis logic", "louis lortie", "love and pitbulls", "love like blood", "love unlimited", "lovedrug", "loverboy", "lovespirals", "low pop suicide", "low roar", "lowe", "lower definition", "lower dens", "lowkey", "lp", "luba", "luca turilli", "luciano", "lucille bogan", "lucinda williams", "lucius", "lucky boys confusion", "lucky dube", "lucy rose", "lucy woodward", "ludacris", "ludo", "ludwig van beethoven", "luigi boccherini", "luka bloom", "lukas graham", "lukas nelson & promise of the real", "luke bryan", "luke doucet", "luke sital-singh", "lullacry", "luna", "lunatica", "lunik", "lupe fiasco", "luscious jackson", "luther allison", "luther vandross", "luv'", "luxt", "lydia", "lydia lunch", "lyfe jennings", "lykke li", "lyle lovett", "lynch mob", "lynn anderson", "lynyrd skynyrd", "lyria", "lyriel", "m (uk)", "m people", "m-flo", "m. ward", "m2m", "m83", "ma rainey", "ma$e", "maanam", "maaya sakamoto", "mac", "mac davis", "mac demarco", "mac miller", "macabre", "machinae supremacy", "machine gun kelly", "machine head", "machine men", "machinemade god", "machines of loving grace", "macklemore", "macklemore & ryan lewis", "macy gray", "mad caddies", "mad marge and the stonecutters", "mad sin", "madchild", "madder mortem", "maddy prior", "madeleine peyroux", "madness", "madonna", "madvillain", "mae", "maestro fresh-wes", "magellan", "maggie reilly", "maggie rogers", "maggie rose", "magic man", "magica", "magna-fi", "magnapop", "magneta lane", "magnum", "maher zain", "mai kuraki", "mainstay", "maisey rika", "majesty", "majid jordan", "major accident", "major parkinson", "make do and mend", "make them suffer", "malcolm holcombe", "malcolm middleton", "malevolent creation", "mali music", "malia", "malinky", "malvina reynolds", "mama cass", "mamas gun", "man", "man overboard", "mance lipscomb", "mandalay", "mandisa", "mandolin orange", "mandragora scream", "mandy barnett", "mandy moore", "manfred mann", "manfred mann's earth band", "manowar", "manticora", "manuel de falla", "maps", "maps & atlases", "marah", "marble sounds", "marc almond", "marc and the mambas", "marc andr hamelin", "marc anthony", "marc cohn", "marc e. bassy", "marcelle meyer", "marcia ball", "marcia griffiths", "marcus orelias", "marduk", "maren ord", "margo price", "margot & the nuclear so and so's", "maria jo o pires", "maria mena", "maria muldaur", "maria solheim", "mariah carey", "marian hill", "marianas trench", "marike jager", "marillion", "marilyn manson", "marina and the diamonds", "mario", "mario winans", "marion", "marion raven", "marissa nadler", "marit larsen", "maritime", "marié digby", "mark chesnutt", "mark collie", "mark eitzel", "mark erelli", "mark harris", "mark heard", "mark king", "mark knopfler", "mark knopfler & emmylou harris", "mark kozelek", "mark owen", "mark ronson", "mark seymour & the undertow", "mark spiro", "markéta irglová", "marlango", "marlene dietrich", "marlon roudette", "marlon williams", "marmaduke duke", "marnie", "maroon", "mars argo", "mars ill", "marsha ambrosius", "martha and the muffins", "martha and the vandellas", "martha argerich", "martha tilston", "martha wainwright", "martika", "martin carthy", "martin carthy and dave swarbrick", "martin jondo", "martin page", "martin zellar", "martina mcbride", "marty robbins", "marty willson-piper", "martyr", "martyr defiled", "marvin gaye", "marvin gaye & tammi terrell", "mary gauthier", "mary hopkin", "mary j. blige", "mary lou lord", "mary mary", "mason jennings", "mason proper", "massacration", "massive attack", "massive ego", "masta ace", "masta killa", "mastedon", "master p", "masterboy", "mastercastle", "masterplan", "masters of reality", "mastodon", "mat kearney", "matchbook romance", "material issue", "mates of state", "math and physics club", "matisyahu", "matt and kim", "matt andersen", "matt bianco", "matt cardle", "matt costa", "matt duke", "matt dusk", "matt goss", "matt haimovitz", "matt maher", "matt mays", "matt nathanson", "matt pond pa", "matt redman", "mattafix", "matthew barber", "matthew dear", "matthew fisher", "matthew friedberger", "matthew good", "matthew herbert", "matthew logan vasquez", "matthew ryan", "matthew sweet and susanna hoffs", "maura o'connell", "maureen mcgovern", "maurice ravel", "maurizio pollini", "mavis staples", "max romeo", "max webster", "maxnormal.tv", "maxwell", "mayer hawthorne", "mayhem", "maylene and the sons of disaster", "maysa leak", "mc chris", "mc eiht", "mc frontalot", "mc hammer", "mc magic", "mc ren", "mc shan", "mc zulu", "mc5", "mcauley schenker group", "mcfly", "mckinney's cotton pickers", "me first and the gimme gimmes", "meat beat manifesto", "meat loaf", "mechanical moth", "mechanical poet", "medina", "mediæval bæbes", "meek mill", "meg & dia", "megadeth", "megafaun", "meiko", "meja", "mekong delta", "mel tillis", "mel tormé", "melanie", "melanie b", "melanie c", "melanie doane", "melanie thornton", "meliah rage", "melissa etheridge", "melissa manchester", "mellowhype", "melody gardot", "melvins", "memoryhouse", "memphis jug band", "memphis may fire", "memphis minnie", "memphis slim", "memphis slim & willie dixon", "memphis willie b.", "men at work", "mendeed", "mennen", "menomena", "mental as anything", "mentallo & the fixer", "menudo", "mercyful fate", "mercyme", "meredith andrews", "merle haggard", "merle travis", "meryn cadell", "meshuggah", "mesmerize", "metal church", "metallica", "metanoia", "method man", "methyl ethel", "metric", "metronomy", "mew", "mewithoutyou", "mf doom", "mgmt", "mgła", "mia doi todd", "mia x", "miasmal", "michael bolton", "michael bublé", "michael card", "michael christmas", "michael franks", "michael hedges", "michael jackson", "michael johnson", "michael kiske", "michael learns to rock", "michael martin murphey", "michael mcdonald", "michael monroe", "michael nesmith", "michael o'brien", "michael rabin", "michael roe", "michael schenker group", "michael schulte", "michael sembello", "michael sweet", "michael w. smith", "michelle branch", "michelle malone", "michelle williams", "michelle wright", "mick jagger", "mick ronson", "mick taylor", "mickey gilley", "mickey newbury", "micky & the motorcars", "microwave dave & the nukes", "midge ure", "midlake", "midnattsol", "midnight juggernauts", "midori goto", "mighty sparrow", "migos", "miguel", "miguel bosé", "miike snow", "mika", "mike & the mechanics", "mike batt", "mike doughty", "mike garrigan", "mike heron", "mike jones", "mike ness", "mike oldfield", "mike posner", "mike scott", "mikhail glinka", "mikhail pletnev", "miki howard", "milburn", "mildred bailey", "milemarker", "miles kane", "miley cyrus", "milk inc.", "milla jovovich", "millencolin", "million dead", "millionaires", "milow", "mimicking birds", "mina", "mind funk", "mind's eye", "mind.in.a.box", "mindless faith", "mindless self indulgence", "mindy smith", "mini mansions", "miniature tigers", "ministry", "mink deville", "minnie driver", "minor threat", "mint condition", "minus story", "minus the bear", "mipso", "mira", "miracle of sound", "mirah", "miranda cosgrove", "miranda lambert", "miranda sex garden", "mirror of deception", "misanthrope", "miseration", "misery index", "misery loves co.", "misfits", "mishka", "miss black america", "miss may i", "miss montreal", "missing persons", "mission of burma", "mississippi fred mcdowell", "mississippi john hurt", "mississippi sheiks", "missy elliott", "missy higgins", "mister monster", "mitchel musso", "mnek", "mnemic", "mo b. dick", "mobb deep", "moby", "moby grape", "moddi", "modern english", "modern romance", "modern skirts", "modern talking", "modest petrovich mussorgsky", "modwheelmood", "mogg way", "mojave 3", "molly hatchet", "molly johnson", "molly venter", "moloko", "molotov solution", "moneybrother", "mono inc.", "monrose", "monster magnet", "monstrosity", "montgomery gentry", "montrose", "monty python", "monuments", "moon martin", "moonface", "moonspell", "morandi", "morbid angel", "morcheeba", "mordacious", "mordred", "more machine than man", "more than life", "morgana lefay", "moriz rosenthal", "mormon tabernacle choir", "morning glory", "mortal love", "mortal sin", "mortician", "mortification", "mos def", "mose allison", "most precious blood", "mostly autumn", "mother love bone", "mother mother", "mothers", "motionless in white", "motorpsycho", "motosierra", "mott the hoople", "motörhead", "moving mountains", "moxy früvous", "moya brennan", "mr weebl", "mr. 3-2", "mr. big", "mr. bungle", "mr. mister", "mr. president", "mr. vegas", "ms. dynamite", "mu330", "much the same", "mud", "muddy waters", "mudvayne", "mugison", "mull historical society", "mumakil", "mumford & sons", "mungo jerry", "municipal waste", "mura masa", "murderdolls", "murray mclauchlan", "murray perahia", "murs", "mushroomhead", "musiq soulchild", "mustard plug", "mustasch", "mutual benefit", "mxpx", "my bloody valentine", "my brightest diamond", "my darkest days", "my favorite", "my life with the thrill kill kult", "my morning jacket", "my passion", "my sister's machine", "my ticket home", "mychildren mybride", "mygrain", "mylon lefevre", "myra hess", "mystery", "mystery skulls", "mystic circle", "mystic prophecy", "mystikal", "máni svavarsson & magnús scheving", "mörk gryning", "mötley crüe", "mø", "múm", "münchener freiheit", "mýa", "n-dubz", "n-trance", "n.e.r.d", "n.w.a", "nadeah", "nahemah", "naia izumi", "nails", "naked aggression", "naked eyes", "namie amuro", "nana", "nana grizol", "nana mouskouri", "nanci griffith", "nancy lamott", "nancy sinatra", "nancy wilson", "naomi", "napalm death", "nappy roots", "narada michael walden", "nargaroth", "narnia", "nas", "nash & young", "nasty c", "nat & alex wolff", "nat king cole", "nat stuckey", "natalia", "natalia kills", "natalie cole", "natalie grant", "natalie imbruglia", "natasha bedingfield", "natasha thomas", "nathan milstein", "nathaniel rateliff", "natural", "nausea", "nazareth", "nazz", "naâman", "nb ridaz", "ne-yo", "neaera", "neal morse", "nebelhexë", "necro", "necrodeath", "necromantia", "negative", "negative approach", "negativland", "neil cicierega", "neil diamond", "neil halstead", "neil sedaka", "neil young", "neko case", "nekromantix", "nektar", "nellie mckay", "nelly", "nelly furtado", "nelson", "nelson freire", "nena", "neneh cherry", "neon hitch", "neon horse", "neon indian", "neon synthesis", "nerina pallot", "nero", "neuraxis", "neuroactive", "neurosis", "neurotech", "neuroticfish", "neva dinova", "never heard of it", "never shout never", "neverending white lights", "nevermore", "nevertheless", "neville marriner", "new boyz", "new edition", "new found glory", "new model army", "new order", "new riders of the purple sage", "new trolls", "new years day", "new york dolls", "new york philharmonic", "newsboys", "newton faulkner", "newworldson", "nf", "nic jones", "niccol paganini", "nick drake", "nick heyward", "nick jonas", "nick kamen", "nick lachey", "nick lowe", "nickel creek", "nickelback", "nicki minaj", "nico", "nicola benedetti", "nicole c. mullen", "nicole dollanganger", "nicole scherzinger", "nicolette larson", "night in gales", "night ranger", "nightmare of you", "nightrage", "nightwish", "nik kershaw", "nikka costa", "nikki flores", "nikki webster", "nikki yanofsky", "nikolai medtner", "nikolai rimsky korsakov", "nile", "nils lofgren", "nim vind", "nina", "nina hagen", "nina kinert", "nina nastasia", "nina nesbitt", "nina simone", "nina sky", "nine", "nine below zero", "nine inch nails", "nine lashes", "ninety pound wuss", "ninja sex party", "nirvana", "nitronoise", "nitty gritty dirt band", "nitzer ebb", "nivea", "nlt", "nneka", "no doctors", "no fun at all", "no knife", "no mercy", "no trend", "no use for a name", "no-man", "noa", "noah gundersen", "nocturnal rites", "nocturne", "noe venable", "nofx", "noggin toboggan", "nolongerhuman", "nomad", "nombe", "nomeansno", "nominon", "non phixion", "noname", "nonpoint", "norah jones", "norma jean", "north mississippi allstars", "north star", "norther", "northern kings", "nothingface", "nouvelle vague", "novembre", "now", "now", "nox arcana", "noxious emotion", "noël coward", "nrbq", "nuclear assault", "nujabes", "numb", "number one gun", "nunslaughter", "oathbreaker", "obey the brave", "obie trice", "obsc(y)re", "oceano", "oceans ate alaska", "october 31", "october fall", "october project", "odd future", "odetta", "odyssey eurobeat", "of mice & men", "of monsters and men", "of montreal", "of the wand & the moon", "off!", "officer negative", "oh", "oh land", "oh susanna", "ohbijou", "ohgr", "ohio", "ohio players", "oi polloi", "okkervil river", "ol' dirty bastard", "old 97's", "old crow medicine show", "old dominion", "old man gloom", "old man luedecke", "old man's child", "olive", "oliver koletzki", "olivia newton-john", "olivier messiaen", "olly murs", "olympos mons", "omnia", "on broken wings", "one bad pig", "one be lo", "one direction", "one less reason", "one without", "oneiroid psychosis", "only crime", "onslaught", "onyx", "ookla the mok", "oomph!", "opera ix", "operation ivy", "opeth", "optiganally yours", "optimus rhyme", "opus", "orchestral manoeuvres in the dark", "ordinary time", "orenda fink", "orgy", "origin", "originoo gunn clappaz", "orleans", "orphanage", "orphaned land", "osborne brothers", "oscar brand", "oscar peterson", "osi", "oszibarack", "otep", "other lives", "other people", "otis redding", "otis rush", "otis spann", "ottorino respighi", "our last night", "out of eden", "out out", "outkast", "outlandish", "outlawz", "over the rhine", "overcome", "overkill", "owain phyfe", "owen pallett", "owl city", "oxymoron", "oysterband", "ozzy osbourne", "p!nk", "p.m. dawn", "p.o.d.", "p.o.s.", "pablo casals", "pablo de sarasate", "pacewon", "paddy goes to holyhead", "page & plant", "page france", "pages", "pain confessor", "pain of salvation", "painbastard", "palaye royale", "palisades", "paloma faith", "pam tillis", "panda bear", "pandora", "pansy division", "pantera", "pantokrator", "paolo nutini", "papa charlie jackson", "papa roach", "papas fritas", "paper aeroplanes", "paper route", "papercuts", "papermoon", "papoose", "papooz", "parachute", "paradise lost", "paragon", "paris", "parker millsap", "parkway drive", "parliament", "parov stelar", "parry gripp", "parts & labor", "partynextdoor", "pascal rog ", "passenger", "passion pit", "pat benatar", "pat mcgee band", "pat travers", "path of resistance", "pathology", "patricia barber", "patrick stump", "patrick watson", "patsy cline", "patti austin", "patti labelle", "patti page", "patty griffin", "patty larkin", "patty loveless", "patty smyth", "paul & mary", "paul and storm", "paul anka", "paul brady", "paul carrack", "paul colman trio", "paul gilbert", "paul kelly", "paul mccartney", "paul oakenfold", "paul revere and the raiders", "paul robeson", "paul rodgers", "paul simon", "paul tortelier", "paul van dyk", "paul wall", "paul weller", "paul whiteman", "paul wilbur", "paul williams", "paul young", "paula abdul", "paulson", "pavlov's dog", "paw", "peabo bryson", "peace", "peaches", "peaches & herb", "pearls before swine", "peccatum", "pedro the lion", "peetie wheatstraw", "peggy lee", "peggy seeger", "penal colony", "pendragon", "pendulum", "penitent", "penny mclean", "pennywise", "pentagram", "pentatonix", "penumbra", "pep love", "pepper", "per gessle", "percy sledge", "periphery", "pernice brothers", "perry como", "persephone", "perzonal war", "pet shop boys", "petal", "pete rock", "pete seeger", "pete shelley", "pete townshend", "peter", "peter & gordon", "peter and the test tube babies", "peter andre", "peter bradley adams", "peter cetera", "peter frampton", "peter gabriel", "peter green", "peter green splinter group", "peter hammill", "peter koppes", "peter serkin", "peter tosh", "peter wolf", "petra", "petula clark", "pfr", "phantom blue", "phantom planet", "pharao", "pharrell williams", "phedora", "phil collins", "phil harris", "phil keaggy", "phil lynott", "phil manzanera", "phil ochs", "phil wickham", "philadelphia orchestra", "philip glass", "philippe entremont", "phillip boa & the voodooclub", "phillip phillips", "phillips", "phinehas", "phobia", "phoebe carrai", "phony ppl", "pianos become the teeth", "piebald", "pierce the veil", "pierre fournier", "pierre laurent aimard", "pietasters", "pieter wispelwey", "pietro locatelli", "pig", "pig destroyer", "pigeon john", "pigface", "pile", "pilot", "pilot speed", "pimp c", "pinback", "pink cream 69", "pink floyd", "pink guy", "pistol annies", "pitboss 2000", "pitbull", "pitchshifter", "pixie lott", "pj morton", "plan b", "planet funk", "planet p project", "planetshakers", "plankeye", "plants and animals", "plasmatics", "play", "player", "playradioplay!", "plumb", "plus one", "plushgun", "pmtoday", "poco", "poison", "poison clan", "poison girls", "poison idea", "poison the well", "poisonblack", "pokey lafarge", "polarkreis 18", "polkadot cadaver", "polluted inheritance", "polly paulusma", "polyenso", "polysics", "poni hoax", "pop etc", "pop evil", "pop unknown", "popa chubby", "popcaan", "poppy", "portastatic", "porter wagoner", "portishead", "portugal. the man", "possessed", "post malone", "poundhound", "power quest", "powerman 5000", "powerwolf", "poxy boggards", "pras", "prefab sprout", "press play", "pretenders", "pretty girls make graves", "pretty ricky", "pretty willie", "pride & glory", "pride and fall", "primal fear", "prime sth", "primer 55", "primus", "prince", "princess nokia", "priscilla ahn", "private line", "pro-pain", "proclamation", "procol harum", "prodigy", "professor green", "project 86", "project pat", "project pitchfork", "promise of redemption", "prong", "propaganda", "propagandhi", "prostitute disfigurement", "protest the hero", "prototype", "provision", "prozak", "prāta vētra", "psalters", "psapp", "pseudo echo", "psy'aviah", "psyche", "psychic tv", "psycho motel", "psychopathic rydas", "psychostick", "psyclon nine", "psycroptic", "public enemy", "public image ltd.", "puffy amiyumi", "puhdys", "pulley", "pulp", "punch brothers", "punchline", "purity ring", "pusha t", "pussycat", "pyotr ilyich tchaikovsky", "pyramaze", "pyrexia", "q-tip", "q5", "qkumba zoo", "qntal", "quarterflash", "quasi", "queen", "queen + paul rodgers", "queensrÿche", "quicksilver messenger service", "quiet riot", "quincy punx", "quinn xcii", "quintorigo", "quo vadis", "quorthon", "r. kelly", "r.a. the rugged man", "r.i.o.", "r.l. burnside", "rabbit junk", "rabia sorda", "racer x", "rachael lampa", "rachael sage", "racoon", "radio birdman", "radney foster", "radu lupu", "rae sremmurd", "raekwon", "raffaella carrà", "raffi", "rag'n'bone man", "rage", "raghav", "raging fyah", "rah digga", "rahsaan patterson", "raimon", "rainbirds", "rainbow", "raintime", "ralph mctell", "ralph stanley", "ralph vaughan williams", "ramblin' jack elliott", "ramones", "ramshackle glory", "rancid", "randy bachman", "randy meisner", "randy newman", "randy rogers band", "randy stonehill", "randy travis", "randy vanwarmer", "raphael saadiq", "rapture", "rare bird", "rare earth", "rasaq", "rascal flatts", "raspberries", "rasputina", "rat boy", "ratcat", "ratt", "raul malo", "raul midón", "raul seixas", "raven-symoné", "ravenous", "ray charles", "ray davies", "ray j", "ray lamontagne", "ray wylie hubbard", "razakel", "razed in black", "rbd", "real estate", "real mccoy", "reamonn", "rebecca lynn howard", "rebellion", "rebelution", "red café", "red fang", "red flag", "red lights flash", "red lorry yellow lorry", "red red meat", "red rider", "red sun rising", "redbone", "redemption", "redgum", "rednex", "redrama", "reel big fish", "reflection eternal", "refused", "reggie and the full effect", "regina spektor", "rehab", "remembering never", "renaissance", "renaldo & the loaf", "renard", "renee olstead", "republica", "resurrection band", "return", "rev theory", "rev. edward w. clayborn", "revamp", "reverend bizarre", "reverend gary davis", "reverend horton heat", "revocation", "revolting cocks", "revolver", "rex goudie", "rex orange county", "rhapsody of fire", "rheostatics", "rhett akins", "rhett miller", "rhiannon giddens", "rhodes", "rhonda vincent", "rhye", "rhythms del mundo", "ria mae", "rialto", "rich boy", "rich kids on lsd", "rich mullins", "richard & linda thompson", "richard buckner", "richard goode", "richard o'brien", "richard shindell", "richard smallwood", "richard strauss", "richard swift", "richard thompson", "richard wagner", "richie furay", "richie havens", "richie kotzen", "richie sambora", "richie spice", "richmond fontaine", "rick astley", "rick derringer", "rick james", "rick ross", "rick springfield", "rick wakeman", "ricky martin", "ricky nelson", "ricky skaggs", "rie fu", "right away", "right said fred", "rihanna", "ring of fire", "ringo starr", "ringworm", "riot", "rise against", "rita connolly", "rita coolidge", "rita ora", "rita springer", "rita wilson", "ritchie valens", "ritual", "rival sons", "river whyless", "rivers of nihil", "riverside", "rjd2", "rob crow", "rob rock", "rob thomas", "robben ford", "robbie fulks", "robbie nevil", "robbie robertson", "robbie williams", "robby valentine", "robert bradley's blackwater surprise", "robert calvert", "robert casadesus", "robert cray", "robert earl keen", "robert ellis", "robert forster", "robert gordon", "robert johnson", "robert palmer", "robert plant", "robert pollard", "robert randolph & the family band", "robert schuman", "robert wyatt", "roberta flack", "robin beck", "robin gibb", "robin trower", "robin williamson", "robin zander", "robyn", "roch voisine", "rockapella", "rocking chairs", "rocky loves emily", "rocky votolato", "rod stewart", "rodney atkins", "rodney crowell", "rodriguez", "roger", "roger chapman", "roger clyne & the peacemakers", "roger daltrey", "roger glover", "roger mcguinn", "roger miller", "roger miret and the disasters", "roger waters", "roger whittaker", "roine stolt", "roko", "roland grapow", "rome", "romeo", "ron hawkins", "ron kenoly", "ron sexsmith", "ron wood", "ronan keating", "ronna reeves", "ronnie dio & the prophets", "ronnie dunn", "ronnie milsap", "roo panes", "room eleven", "roomful of blues", "roosevelt sykes", "rooster", "roots manuva", "rootwater", "rorschach test", "rory block", "rory gallagher", "rosaline", "rosalyn tureck", "rosanne cash", "rose cousins", "rose funeral", "rose maddox", "rose polenzani", "rose royce", "rosemary clooney", "rosetta stone", "rosie thomas", "ross lynch", "rotersand", "rotten sound", "rotting christ", "rowland s. howard", "roxette", "roxy music", "roy acuff", "roy buchanan", "roy clark", "roy drusky", "roy orbison", "roy woods", "royal hunt", "royal trux", "royal tusk", "royal wood", "ruby", "rudimental", "rudimentary peni", "rudolf firku n ", "rudolf serkin", "rudy vallée", "rufio", "rufus", "rufus thomas", "rufus wainwright", "rumer", "run kid run", "run level zero", "run the jewels", "run-d.m.c.", "runemagick", "running wild", "runrig", "rupaul", "rupert hine", "rush", "russ", "russ taff", "russian red", "rusted root", "ruston kelly", "ry cooder", "ryan adams", "ryan adams and the cardinals", "ryan bingham", "ryan delmore", "ryan shupe & the rubberband", "ryan stevenson", "ryker's", "rza", "róisín murphy", "röyksopp", "rüfüs du sol", "s", "s club 7", "s.f.a.", "s.l.a.b.", "s.o.a.p.", "s.p.o.c.k", "sabaton", "sabbat", "sabrina", "sabrina carpenter", "sabrina claudio", "sabrina starke", "sacred warrior", "sad café", "sadat x", "sade", "sadist", "sadistik", "saga", "sage francis", "sahara hotnights", "said the whale", "saidian", "saint etienne", "saint lu", "saint motel", "saint raymond", "saint saviour", "sally oldfield", "sally seltmann", "salt the wound", "salt-n-pepa", "salvador", "sam & dave", "sam brown", "sam cooke", "sam lewis", "sam phillips", "sam smith", "sam the sham & the pharaohs", "sam tsui", "samael", "samantha fox", "sambassadeur", "samhain", "sammie", "sammy adams", "sammy davis jr.", "sammy hagar", "samson", "samson fran ois", "samuel barber", "san cisco", "san francisco symphony", "sananda maitreya", "sanchez", "sanctus real", "sandi thom", "sandie shaw", "sandra", "sandy & junior", "sandy denny", "santana", "sara bareilles", "sara evans", "sara gazarek", "sara k.", "sara lov", "sara noxx", "sara watkins", "sarah blasko", "sarah brightman", "sarah connor", "sarah darling", "sarah harmer", "sarah jarosz", "sarah mclachlan", "sarah vaughan", "saraya", "sarina paris", "sarke", "sash!", "sasha", "satan", "satellites", "saturday looks good to me", "saucy monky", "saukrates", "saul williams", "savage garden", "savatage", "save ferris", "saves the day", "saving abel", "saving jane", "saviour machine", "savoir adore", "savoy brown", "sawyer brown", "saxon", "saywecanfly", "scala & kolacny brothers", "scandroid", "scanner", "scapegoat", "scar symmetry", "scarface", "scarling.", "scarve", "scary bitches", "scary kids scaring kids", "scatman john", "schiller", "school of seven bells", "schoolboy q", "schäffer the darklord", "scooter", "scorpions", "scott bradlee's postmodern jukebox", "scott joplin", "scott matthew", "scott matthews", "scott mckenzie", "scott miller", "scouting for girls", "screaming mechanical brain", "screaming trees", "screeching weasel", "screwed up click", "scritti politti", "scud mountain boys", "scythe", "sea of treachery", "sea wolf", "seabear", "seabound", "seahaven", "seal", "seals & crofts", "seam", "sean paul", "sean watkins", "search the city", "seasick steve", "sebadoh", "sebastian sturm", "second person", "secondhand serenade", "secret army", "secret garden", "secret lives of the freemasons", "secrets", "secrets of the moon", "see you next tuesday", "seeed", "selah sue", "selena", "selena gomez", "selena gomez & the scene", "sense field", "senses fail", "sentenced", "seraphim shock", "serena ryder", "sergei prokofiev", "sergei rachmaninoff", "sesame street", "set your goals", "seth lakeman", "seth walker", "sevdaliza", "seven places", "sevendust", "seventh avenue", "seventh day slumber", "sevyn streeter", "sex gang children", "sex pistols", "sexy sadie", "shabazz the disciple", "shad", "shadow gallery", "shadowland", "shadows fall", "shadowside", "shaggy", "shaggy 2 dope", "shai hulud", "shai linne", "shakespears sister", "shakey graves", "shakin' stevens", "shakira", "shalamar", "sham 69", "shamir", "shampoo", "shane & shane", "shane barnard", "shania twain", "shanice", "shannon & the clams", "shannon curfman", "shark island", "sharon jones & the dap-kings", "sharon needles", "sharon van etten", "shaun cassidy", "shawn colvin", "shawn desman", "shawn james", "shawn mendes", "shaye", "she & him", "she keeps bees", "she said", "she wants revenge", "shearer", "shearwater", "sheavy", "sheb wooley", "shedaisy", "sheena easton", "sheila e.", "shel silverstein", "shelby lynne", "shellac", "shelley fabares", "shenandoah", "sheppard", "sherban lupu", "sherbet", "sheryl crow", "shimshai", "shinee", "shining", "shirley bassey", "shirley caesar", "shirley horn", "shlomo mintz", "sho baraka", "shocking blue", "shockwave", "shoffy", "shola ama", "shooter jennings", "short stack", "shotgun messiah", "shout out louds", "showaddywaddy", "showbread", "shudder to think", "shura cherkassky", "shwayze", "shy", "shy girls", "shyne", "sick of it all", "sick of sarah", "sieges even", "sigh", "sigrid", "silent civilian", "silent cry", "silent stream of godless elegy", "silkk the shocker", "silly wizard", "silver convention", "silver jews", "silverstein", "simon and garfunkel", "simple minds", "simply red", "since october", "sinergy", "singer", "sinheresy", "sinister", "sinner", "sinéad lohan", "siobhan donaghy", "sir mix-a-lot", "sirenia", "sirus", "sister hazel", "sister machine gun", "sister sin", "sister sledge", "sita", "sitd", "sitti", "siva six", "sivert høyem", "six feet under", "skeeter davis", "skeleton key", "skepticism", "skew siskin", "skid row", "skiltron", "skinless", "skinny puppy", "skip james", "skip the use", "skold", "skrew", "skrewdriver", "skrillex", "skyclad", "skydiggers", "skye", "skye sweetnam", "skyharbor", "skylark", "slade", "slapp happy", "slapshot", "slash", "slash's snakepit", "slaughter", "slaves on dope", "slayer", "slechtvalk", "sleep", "sleeper", "sleeping giant", "sleeping with sirens", "sleepy john estes", "sleigh bells", "slice the cake", "slick rick", "slightly stoopid", "slim", "slim cessna's auto club", "slim dusty", "slim gaillard", "slim harpo", "slim thug", "slingshot dakota", "slobberbone", "slowdive", "slugdge", "sly & the family stone", "small faces", "smile.dk", "smog", "smokey robinson", "smokey robinson & the miracles", "smokie", "smokin' joe kubek & bnois king", "smoking popes", "smooth", "smosh", "smp", "snafu", "snap!", "sneaker pimps", "snfu", "snog", "snoop dogg", "snot", "snow", "snow tha product", "snowy white", "so many dynamos", "sodom", "soen", "sofie", "soft machine", "sohn", "soja", "soko", "sol gabetta", "sol invictus", "sole", "solefald", "solitude aeturnus", "solomon burke", "solomon cutner", "someone still loves you boris yeltsin", "something corporate", "something with numbers", "son house", "son lux", "son volt", "sonata arctica", "sondre lerche", "sonia disappearfear", "sonicflood", "sonny & cher", "sonny boy williamson i", "sonny boy williamson ii", "sonny james", "sonny landreth", "sonny terry", "sonny terry & brownie mcghee", "sonreal", "sons of bill", "sons of butcher", "sons of seasons", "sons of the pioneers", "sonya kitchell", "sophie ellis-bextor", "sophie zelmani", "sopor aeternus", "soul embraced", "soul position", "souldecision", "soulfly", "soulja boy", "soulsavers", "soulspell", "sound tesselated", "south border", "south park", "southern culture on the skids", "southern raiders band", "southgang", "southside johnny & the asbury jukes", "spacemen 3", "spahn ranch", "spain", "spandau ballet", "spank rock", "sparks", "spazz", "speaker", "speaker", "spectral", "spellblast", "spheric universe experience", "spice 1", "spice girls", "spike jones", "spin doctors", "spinal tap", "spirit", "spirit of the west", "spiritual front", "spirogyra", "spitalfield", "splashdown", "split enz", "splitsville", "spock's beard", "spoon", "spoons", "squealer", "squeeze", "st. vincent", "stabilo", "stacey kent", "stacey q", "stacie orrico", "stacy lattisaw", "stage dolls", "stan freberg", "stan ridgway", "stan rogers", "stanfour", "star fucking hipsters", "star one", "starbenders", "starbomb", "starfield", "starflyer 59", "starlight mints", "stars", "starship", "state property", "status quo", "stauros", "stavesacre", "stealers wheel", "steam powered giraffe", "steel attack", "steel train", "steeler", "steeleye span", "steelheart", "steelwing", "steely dan", "steep", "stefanie heinzmann", "stellastarr*", "stemm", "stepdad", "stephan eicher", "stephanie mills", "stephen bishop", "stephen duffy", "stephen fretwell", "stephen hough", "stephen lynch", "stephen malkmus", "stephen marley", "stephen schwartz", "stephen sondheim", "stephen stills", "stephin merritt", "steppenwolf", "steps", "stereo skyline", "stereolab", "steril", "stetsasonic", "steve aoki", "steve carlson", "steve earle", "steve forbert", "steve goodman", "steve grand", "steve green", "steve hackett", "steve harley & cockney rebel", "steve hillage", "steve holy", "steve mason", "steve mcconnell", "steve miller band", "steve moakler", "steve perry", "steve taylor", "steve vai", "steve wariner", "steve winwood", "steven curtis chapman", "steven isserlis", "steven wilson", "stevie b", "stevie nicks", "stevie ray vaughan", "stevie wonder", "stick to your guns", "sticky fingaz", "stiff little fingers", "still remains", "stillborn", "stills", "stills & nash", "sting", "stompin' tom connors", "stone breath", "stonewall jackson", "stormtroopers of death", "stormwarrior", "stornoway", "straight faced", "strand of oaks", "strange majik", "strapping young lad", "stratovarius", "strawberry alarm clock", "strawbs", "stray cats", "straylight run", "stream of passion", "streetheart", "streetlight manifesto", "streetwalkers", "stretch", "strfkr", "strike anywhere", "stromkern", "structures", "stryper", "sturgill simpson", "stutterfly", "styles p", "styx", "sub focus", "sub7even", "sublime", "suburban legends", "sufjan stevens", "sugababes", "sugarland", "suggs", "suicidal angels", "suicidal tendencies", "suicide commando", "suicide silence", "suidakra", "summoning", "sun kil moon", "sundara karma", "sunday's best", "sunset rubdown", "super junior", "super junior-d&e", "superbus", "superchick", "superjoint ritual", "supersuckers", "supertramp", "surface", "survivor", "susan ashton", "susan boyle", "susan raye", "susan tedeschi", "susanna hoffs", "susannah mccorkle", "suspyre", "suzanne vega", "suzi quatro", "suzy bogguss", "sviatoslav richter", "swans", "sweat & tears", "sweatshop union", "sweet", "sweet comfort band", "sweet noise", "sweetbox", "swim deep", "swing out sister", "swingin' utters", "swizz beatz", "swmrs", "swollen members", "sworn enemy", "swv", "sybreed", "syd barrett", "syd matters", "syleena johnson", "sylosis", "sylvan esso", "sylver", "symbols", "sympathy", "symphony x", "syreeta", "system syn", "sérgio mendes", "sóley", "t'pau", "t-bone", "t-bone walker", "t-pain", "t. mills", "t. rex", "t.a.t.u.", "t.i.", "t.j. miller", "tab benoit", "taco", "tactical sekt", "tad morose", "tahiti 80", "tait", "taj mahal", "taj weekes & adowa", "takara", "take 6", "take that", "taken by trees", "talib kweli", "talisco", "talisman", "talk talk", "talking heads", "tall dwarfs", "tame impala", "tammi terrell", "tammy wynette", "tampa red", "tanita tikaram", "tank", "tankard", "tanya tucker", "tape five", "tapping the vein", "tara maclean", "tarja", "tarkio", "tarnation", "tarot", "taylor dayne", "taylor swift", "teacup monster", "team dresch", "team starkid", "tear da club up thugs", "tears for fears", "tech n9ne", "ted leo and the pharmacists", "ted nugent", "tedashii", "teddy pendergrass", "tedeschi trucks band", "teen suicide", "teena marie", "television", "ten masked men", "ten years after", "tender", "tender forever", "tennessee ernie ford", "tenth avenue north", "teodasia", "terence trent d'arby", "teresa brewer", "terminal choice", "terri clark", "terror jr", "terror squad", "terrorgruppe", "terry callier", "terry reid", "tesseract", "testament", "tex ritter", "tex williams", "texas in july", "teyana taylor", "tha dogg pound", "thalía", "thank you scientist", "that handsome devil", "that petrol emotion", "thaurorod", "the 3rd and the mortal", "the 5th dimension", "the 69 eyes", "the 88", "the acacia strain", "the academy is...", "the adicts", "the afters", "the agonist", "the agony scene", "the alan parsons project", "the allman brothers band", "the almanac singers", "the almighty", "the aluminum group", "the amazing rhythm aces", "the ambassador", "the amboy dukes", "the ames brothers", "the amity affliction", "the andrews sisters", "the angels", "the angels of light", "the animals", "the answer", "the antlers", "the appleseed cast", "the aquabats!", "the archies", "the arka teks", "the arrogant sons of bitches", "the arrogant worms", "the associates", "the association", "the audition", "the autumn offering", "the avett brothers", "the b-52's", "the babys", "the bacon brothers", "the bad shepherds", "the badlees", "the ballroom thieves", "the band", "the band perry", "the bangles", "the bar-kays", "the barr brothers", "the baseballs", "the be good tanyas", "the beach boys", "the bears", "the beatles", "the beatnuts", "the beau brummels", "the beautiful south", "the bellamy brothers", "the bellrays", "the beloved", "the berzerker", "the beta band", "the bianca story", "the big pink", "the birthday massacre", "the birthday party", "the black angels", "the black crowes", "the black dahlia murder", "the black eyed peas", "the black keys", "the black lillies", "the blasters", "the bled", "the blind boys of alabama", "the blow", "the blow monkeys", "the blues brothers", "the bobs", "the bolshoi", "the bongos", "the bonzo dog doo-dah band", "the boo radleys", "the books", "the boomtown rats", "the boswell sisters", "the bottle rockets", "the bouncing souls", "the box tops", "the boxmasters", "the boy least likely to", "the boys", "the boyz", "the brand new heavies", "the breather", "the brian setzer orchestra", "the broken family band", "the bronx", "the brothers four", "the brothers johnson", "the browning", "the browns", "the bruisers", "the brunettes", "the buckinghams", "the buffoons", "the bunny the bear", "the burning hell", "the burns sisters", "the business", "the busters", "the butchies", "the byrds", "the call", "the cars", "the carter family", "the casket lottery", "the casting out", "the casualties", "the cave singers", "the chain gang of 1974", "the chainsmokers", "the chameleons", "the chap", "the chariot", "the charlie daniels band", "the chasm", "the cheeky girls", "the cheetah girls", "the chemical brothers", "the chieftains", "the chiffons", "the choir", "the chordettes", "the church", "the city harmonic", "the civil wars", "the clancy brothers", "the clash", "the classic crime", "the clay people", "the clientele", "the cliks", "the clovers", "the coathangers", "the cog is dead", "the color changin' click", "the color morale", "the colourfield", "the commitments", "the communards", "the concretes", "the constructus corporation", "the contortionist", "the cooper temple clause", "the corries", "the corrs", "the coup", "the courteeners", "the cover girls", "the crabb family", "the cramps", "the creator", "the creepshow", "the crest", "the cross", "the cross movement", "the crown", "the crucified", "the cryan' shames", "the crystal method", "the crüxshadows", "the cyrkle", "the czars", "the damned", "the darkest of the hillside thickets", "the darkness", "the datsuns", "the dayton family", "the db's", "the dc3", "the dears", "the deep dark woods", "the del mccoury band", "the delgados", "the delmore brothers", "the derek trucks band", "the desert sessions", "the devil makes three", "the devil wears prada", "the dickies", "the dignity of labour", "the dillards", "the dillinger escape plan", "the dingees", "the diplomats", "the dismemberment plan", "the divine comedy", "the doc watson family", "the dodos", "the dogma", "the dollyrots", "the donefors", "the donnas", "the doobie brothers", "the doors", "the dreadnoughts", "the dreaming", "the drifters", "the drums", "the dubliners", "the duckworth lewis method", "the ducky boys", "the duhks", "the durutti column", "the duskfall", "the dø", "the echoing green", "the elected", "the electric hellfire club", "the electric prunes", "the enemy", "the english beat", "the essex green", "the everly brothers", "the ex", "the expos", "the eyes of a traitor", "the fabulous thunderbirds", "the faceless", "the faction", "the faint", "the fair sex", "the fall of troy", "the feelies", "the felice brothers", "the fiery furnaces", "the fixx", "the flashbulb", "the flatliners", "the flight of sleipnir", "the floacist", "the flower kings", "the fold", "the forecast", "the foreign exchange", "the foreshadowing", "the forgotten rebels", "the format", "the foundations", "the four aces", "the four freshmen", "the four lads", "the four seasons", "the frames", "the free design", "the friday night boys", "the front bottoms", "the fullblast", "the funeral pyre", "the futureheads", "the gabe dixon band", "the gadjits", "the game", "the gap band", "the gathering", "the georgia satellites", "the geraldine fibbers", "the germs", "the ghost inside", "the ghost of a saber tooth tiger", "the glorious unseen", "the go! team", "the go-betweens", "the go-go's", "the good life", "the gothic archies", "the gothsicles", "the gourds", "the grapes of wrath", "the grass roots", "the grates", "the gray havens", "the gregory brothers", "the grouch", "the groundhogs", "the guess who", "the gufs", "the gun club", "the halliard", "the handsome family", "the hard-ons", "the haunted", "the head and the heart", "the heavy", "the helio sequence", "the hellacopters", "the henry girls", "the herd", "the hidden cameras", "the high dials", "the higher", "the highwaymen", "the hippos", "the hold steady", "the hollies", "the honorary title", "the hooters", "the horrors", "the housemartins", "the human abstract", "the human league", "the hundred in the hands", "the hunna", "the icicle works", "the idle race", "the impossibles", "the impressions", "the incredible string band", "the indelicates", "the ink spots", "the insyderz", "the internet", "the interrupters", "the irish rovers", "the isley brothers", "the j. geils band", "the jackson 5", "the jam", "the japanese house", "the jayhawks", "the jim yoshii pile-up", "the jimi hendrix experience", "the joe perry project", "the jon spencer blues explosion", "the jordanaires", "the joy formidable", "the judds", "the juliana theory", "the kelly family", "the kentucky headhunters", "the kids from fame", "the kills", "the kings", "the kingston trio", "the kinks", "the klezmatics", "the klf", "the knack", "the knife", "the kooks", "the kovenant", "the kry", "the l-train", "the lads", "the last bison", "the last dance", "the last shadow puppets", "the laurie berkner band", "the lawrence arms", "the left banke", "the left rights", "the letter black", "the limeliters", "the limousines", "the little willies", "the locust", "the lone bellow", "the lonely forest", "the lonely island", "the long blondes", "the long winters", "the lookouts", "the lords of the new church", "the lost trailers", "the loud family", "the louvin brothers", "the loved ones", "the lovin' spoonful", "the low anthem", "the lox", "the lucksmiths", "the lumineers", "the lyric quartet", "the maccabees", "the mad conductor", "the magic numbers", "the magnetic fields", "the maine", "the malibooz", "the mamas & the papas", "the manhattan transfer", "the manhattans", "the maranatha! singers", "the mars volta", "the marvelettes", "the mary onettes", "the matches", "the mavericks", "the mayan factor", "the mccalmans", "the meads of asphodel", "the megas", "the mendoza line", "the menzingers", "the merry wives of windsor", "the meteors", "the methadones", "the microphones", "the midnight beast", "the mighty mighty bosstones", "the million dollar quartet", "the miracles", "the mission", "the mistake", "the moffatts", "the moldy peaches", "the monkees", "the monochrome set", "the moody blues", "the moog", "the more i see", "the morning of", "the most serene republic", "the motels", "the move", "the movielife", "the muppets", "the murder city devils", "the murder of my sweet", "the museum", "the mutton birds", "the mynabirds", "the myriad", "the mākaha sons", "the national", "the neighbourhood", "the nerve agents", "the neville brothers", "the new pornographers", "the new power generation", "the nice", "the night flight orchestra", "the nighthawks", "the nightwatchman", "the nits", "the northern pikes", "the notorious b.i.g.", "the number twelve looks like you", "the nylons", "the o.c. supertones", "the oak ridge boys", "the obsessed", "the ocean", "the ocean blue", "the offspring", "the oh hellos", "the oppressed", "the osmonds", "the other", "the other ones", "the outfield", "the pale fountains", "the panic division", "the paper chase", "the partridge family", "the paul butterfield blues band", "the pentangle", "the persuasions", "the pharcyde", "the philosopher kings", "the pierces", "the pillows", "the pineapple thief", "the platters", "the plimsouls", "the plot in you", "the pogues", "the pointer sisters", "the police", "the poodles", "the pop group", "the posies", "the power station", "the presets", "the presidents of the united states of america", "the pretty reckless", "the pretty things", "the prids", "the proclaimers", "the prodigy", "the project hate mcmxcix", "the promise ring", "the psychedelic ensemble", "the psychedelic furs", "the psycho realm", "the pursuit of happiness", "the pussycat dolls", "the quakes", "the raconteurs", "the radio dept.", "the rakes", "the rankin family", "the rapture", "the rascals", "the rasmus", "the raveonettes", "the ready set", "the real mckenzies", "the real tuesday weld", "the reason", "the receiving end of sirens", "the red jumpsuit apparatus", "the red shore", "the refreshments", "the regrettes", "the reign of kindo", "the rembrandts", "the replacements", "the republic of wolves", "the residents", "the retrosic", "the righteous brothers", "the ritchie family", "the riverboat gamblers", "the road hammers", "the roches", "the rolling stones", "the romantics", "the ronettes", "the roots", "the rubettes", "the rugburns", "the rumour said fire", "the runaways", "the rural alberta advantage", "the russian futurists", "the rutles", "the s.o.s. band", "the saints", "the samples", "the saturdays", "the saw doctors", "the scabs", "the scary jokes", "the scene aesthetic", "the screaming jets", "the script", "the searchers", "the seatbelts", "the secret handshake", "the seekers", "the seer", "the sensational alex harvey band", "the servant", "the shadows", "the shamen", "the shangri-las", "the sheepdogs", "the sheila divine", "the shins", "the ship", "the shirelles", "the showdown", "the shroud", "the silencers", "the silver shine", "the sisters of mercy", "the skids", "the sleeping", "the sleepy jackson", "the smashing pumpkins", "the smithereens", "the smiths", "the soft boys", "the sorrow", "the soul of john black", "the soul stirrers", "the sound", "the sound of animals fighting", "the sounds", "the soundtrack of our lives", "the soviettes", "the spencer davis group", "the spill canvas", "the spinners", "the spinto band", "the stanley brothers", "the stereo", "the stills", "the stone roses", "the stooges", "the story", "the story so far", "the stranglers", "the streets", "the string cheese incident", "the strokes", "the strumbellas", "the stryder", "the strypes", "the stupid stupid henchmen", "the style council", "the stylistics", "the submarines", "the suburbs", "the subways", "the sugarcubes", "the suicide file", "the summer set", "the sundays", "the supernaturals", "the supremes", "the supremes & the four tops", "the swell season", "the swellers", "the swift", "the tangent", "the tea party", "the tear garden", "the temperance movement", "the temptations", "the tenors", "the the", "the thermals", "the thrills", "the time", "the ting tings", "the title", "the tony danza tapdance extravaganza", "the tony rich project", "the town pants", "the tragically hip", "the trammps", "the treatment", "the tremeloes", "the trews", "the triffids", "the troggs", "the tubes", "the turtles", "the twang", "the twilight sad", "the twilight singers", "the twins", "the undertones", "the unicorns", "the unseen", "the unthanks", "the used", "the vaccines", "the vamps", "the vandals", "the vapors", "the veils", "the velvet teen", "the velvet underground", "the ventures", "the veronicas", "the verve", "the verve pipe", "the vibrators", "the view", "the vincent black shadow", "the vines", "the violet burning", "the virus", "the voidz", "the waifs", "the wailers", "the wailin' jennys", "the walkabouts", "the walker brothers", "the walkmen", "the wallflowers", "the wannadies", "the wanted", "the war on drugs", "the warning", "the warren brothers", "the watchmen", "the waterboys", "the weakerthans", "the weavers", "the wedding", "the weeknd", "the weepies", "the whispers", "the white birch", "the white buffalo", "the white stripes", "the whitest boy alive", "the who", "the wiggles", "the wildhearts", "the wilkinsons", "the winery dogs", "the wonder stuff", "the wonder years", "the working title", "the world is a beautiful place & i am no longer afraid to die", "the world of skin", "the wrights", "the xx", "the yardbirds", "the yards", "the years gone by", "the youngbloods", "the zolas", "the zombies", "the zutons", "the-dream", "thea gilmore", "theatre of tragedy", "theatres des vampires", "thee oh sees", "thee silver mt. zion", "thelma houston", "them", "theory in practice", "there for tomorrow", "therefore i am", "theresa sokyrka", "therion", "these new puritans", "thestart", "they might be giants", "thi'sl", "thievery corporation", "thieves and villains", "thin lizzy", "thingy", "third day", "third eye blind", "third world", "thirsty merc", "thirteen senses", "thirty seconds to mars", "this beautiful republic", "this bike is a pipe bomb", "this day & age", "this ending", "this is hell", "this is the kit", "this mortal coil", "this providence", "this wild life", "thomas dolby", "thompson twins", "thor", "thornley", "those dancing days", "thou", "thousand foot krutch", "thoushaltnot", "three 6 mafia", "three dog night", "three plus", "threshold", "thrice", "through the eyes of the dead", "throwdown", "thrush hermit", "thumb", "thunder", "thunder lord", "thundercat", "thunderstone", "thundra", "thursday", "thurston moore", "thy art is murder", "thy primordial", "thyrane", "thyx", "tiago iorc", "tiamat", "tiara thomas", "tides of man", "tiffany evans", "tift merritt", "tiga", "tiger army", "tigers jaw", "tila tequila", "tilly and the wall", "tilt", "tim curry", "tim fite", "tim hardin", "tim hughes", "tim kasher", "tim maia", "tim mcgraw", "tim moore", "tim o'brien", "timbaland", "timbaland & magoo", "timber timbre", "timbuk3", "timothy seth avett as darling", "tin machine", "tina arena", "tina charles", "tina dico", "tina guo", "tina turner", "tinashe", "tinchy stryder", "tindersticks", "tinfed", "tinie tempah", "tink", "tiny ruins", "tiny tim", "tism", "titanic sinclair", "title fight", "titus andronicus", "tizzy bac", "tiësto", "tko", "tlc", "tnt", "to die for", "to kill a king", "to speak of wolves", "toad the wet sprocket", "toby keith", "tobymac", "today is the day", "todd rundgren", "todd snider", "toh kay", "tohoshinki", "tokio hotel", "tokyo police club", "tom cochrane", "tom dice", "tom fogerty", "tom jones", "tom lehrer", "tom mcrae", "tom milsom", "tom misch", "tom odell", "tom paxton", "tom petty and the heartbreakers", "tom robinson", "tom robinson band", "tom rosenthal", "tom rush", "tom russell", "tom smith", "tom tom club", "tom vek", "tom waits", "tombs", "tommy castro", "tommy dorsey", "tommy james", "tommy james and the shondells", "tommy lee", "tommy mcclennan", "tommy page", "tommy roe", "tommy shaw", "tomohisa yamashita", "tomorrows bad seeds", "tone damli", "tonedeff", "toni braxton", "toni childs", "tonic", "tonight alive", "tonio k", "tony banks", "tony bennett", "tony iommi", "tony joe white", "tony orlando & dawn", "tony rice", "tony rose", "tony yayo", "tony! toni! toné!", "too $hort", "too bad eugene", "too pure to die", "toots & the maytals", "tor miller", "tori amos", "tori kelly", "toro y moi", "torres", "toto", "tourniquet", "tove lo", "tower of power", "towers of london", "townes van zandt", "toxik", "toy dolls", "toy-box", "toyah", "tq", "tracey thorn", "tracie spencer", "tracy byrd", "tracy chapman", "tracy grammer", "tracy lawrence", "trademark", "trae", "traffic", "tragic black", "trail of tears", "train", "trans-siberian orchestra", "transatlantic", "transit", "transmetal", "transvision vamp", "trapeze", "trashcan sinatras", "travie mccoy", "travis", "travis scott", "treble charger", "tree63", "trespassers william", "trevor rabin", "trey anastasio", "trey songz", "trial", "tribe", "trick daddy", "tricky", "triggerfinger", "trijntje oosterhuis", "trin-i-tee 5", "trip lee", "trip shakespeare", "tripod", "trippie redd", "triptykon", "trisha yearwood", "tristan prettyman", "tristania", "triumph", "triumvirat", "trivium", "trixter", "trocadero", "trooper", "trophy eyes", "trophy scars", "trouble", "trouble over tokyo", "tru", "true colors", "true widow", "truls m rk", "truly", "trust", "truth hurts", "tsjuder", "tsol", "tsunami bomb", "tuatha de danann", "tub ring", "tuck & patti", "tunng", "turin brakes", "turisas", "turk", "turnpike troubadours", "tv on the radio", "tw walsh", "tweet", "twenty one pilots", "twila paris", "twilight fauna", "twilight force", "twilightning", "twin shadow", "twista", "twisted sister", "twiztid", "two hours traffic", "twrp", "ty england", "ty herndon", "ty segall", "tye tribbett", "tyler", "tyler bryant & the shakedown", "tyler lyle", "tyler shaw", "type o negative", "typhoon", "tyrese", "tyrone wells", "tystnaden", "tzu", "týr", "u-god", "u.d.o.", "u.n.l.v.", "u.s. bombs", "u2", "ub40", "udora", "ufo", "ufx", "ugk", "ugly duckling", "ugly kid joe", "ugress", "ulcerate", "ultimate fakebook", "ultimatum", "ultra", "ultraviolet sound", "ultravox", "ulver", "umphrey's mcgee", "unbelievable truth", "uncle acid & the deadbeats", "uncle dave macon", "uncle tupelo", "underoath", "underworld", "unearth", "unearthly trance", "unexpect", "union", "union 13", "union j", "unisonic", "unit", "unitopia", "unkle", "unknown", "unknown hinson", "unknown mortal orchestra", "unloco", "unspoken", "unsung zeros", "unter null", "unwritten law", "upon a burning body", "upon this dawning", "uriah heep", "urma", "urthboy", "us the duo", "us3", "useless id", "usher", "utada hikaru", "vacuum", "vader", "valencia", "valentine wolfe", "valient thorr", "vallenfyre", "vampire rodents", "vampire weekend", "van canto", "van cliburn", "van der graaf generator", "van dyke parks", "van halen", "van hunt", "van morrison", "vance joy", "vanden plas", "vandenberg", "vanessa amorosi", "vanessa bell armstrong", "vanessa carlton", "vanessa hudgens", "vanessa williams", "vangelis", "vangough", "vanilla fudge", "vanilla ice", "vanna", "varsity fanclub", "vashti bunyan", "vast", "vaux", "vaya con dios", "veda hille", "veggietales", "veil of maya", "velvet acid christ", "velvet belly", "velvet revolver", "vendetta", "vendetta red", "venetian princess", "venetian snares", "vengaboys", "vengeance", "vengeance rising", "venom", "venomous concept", "vera lynn", "vern gosdin", "versaemerge", "vertical church band", "vertical horizon", "veruca salt", "vetiver", "veto", "vic chesnutt", "vic damone", "vicious crusade", "vicious rumors", "vicki lawrence", "vicky beeching", "victims family", "victoria beckham", "victory", "vienna philharmonic", "vienna teng", "vigilantes of love", "viktor vaughn", "viktoria mullova", "village people", "vince gill", "vince neil", "vincenzo bellini", "vinnie paz", "vintage trouble", "vinyl theatre", "violent femmes", "violent soho", "violent work of art", "viper", "virgin prunes", "virgin steele", "virtuoso", "vision divine", "visions of atlantis", "viva voce", "vixen", "vladimir ashkenazy", "vladimir horowitz", "vladimir sofronitsky", "vnv nation", "voivod", "voltaire", "volumes", "vomito negro", "vonda shepard", "vortech", "votum", "voxtrot", "vulfpeck", "vv brown", "vázquez sounds", "vérité", "w-inds.", "w.a.s.p.", "wade bowen", "wale", "walk off the earth", "walk the moon", "wall of voodoo", "wallis bird", "wallows", "walls of jericho", "waltari", "walter becker", "walter egan", "walter gieseking", "walter trout", "wanda jackson", "wang chung", "war", "war from a harlots mouth", "war of ages", "warbringer", "warcloud", "warlock", "warpaint", "warrant", "warren haynes", "warren zevon", "washboard sam", "washed out", "watain", "watch tower bible and tract society", "watermark", "waters", "watershed", "wax", "waylon", "waylon jennings", "wayne hancock", "wayne newton", "we are the in crowd", "we as human", "we came as romans", "we five", "we shot the moon", "we the kings", "webb pierce", "webb wilder", "webbie", "wednesday 13", "weedeater", "weekend nachos", "ween", "weeping tile", "weerd science", "weezer", "weh", "wendy matthews", "westlife", "westside connection", "wham!", "when particles collide", "whigfield", "while she sleeps", "whirlwind heat", "whiskeytown", "white denim", "white heart", "white lies", "white lion", "white sea", "whitechapel", "whitecross", "whitehorse", "whitehouse", "whitesnake", "whitney houston", "whores.", "whose line is it anyway? cast", "why don't we", "wide mouth mason", "widespread panic", "widowmaker", "wig wam", "wigwam", "wilco", "wild child", "wild nothing", "wild orchid", "wild strawberries", "wildbirds & peacedrums", "wildpath", "wilhelm backhaus", "wilhelm kempff", "will oldham", "will smith", "will stratton", "will young", "will.i.am", "willam", "willard grant conspiracy", "william beckett", "william elliott whitmore", "william fitzsimmons", "william kapell", "william mcdowell", "william shatner", "willie d", "willie dixon", "willie nelson", "willie nelson & wynton marsalis", "willow smith", "willy deville", "willy porter", "wilson phillips", "wilson pickett", "wind & fire", "winds", "winds of plague", "winger", "wings", "winter's bane", "wintersleep", "winterstorm", "wire", "wise guys", "wishbone ash", "wishing chair", "with confidence", "witherscape", "within reason", "within temptation", "within the ruins", "wizard", "wizards", "woe", "wolf alice", "wolf gang", "wolf parade", "wolfe tones", "wolfgang amadeus mozart", "wolfgang schneiderhan", "wolfgun", "wolfheart", "wolfmother", "wolfpakk", "wolfsheim", "wolfstone", "wolverine", "wonder girls", "wondermints", "wonderwall", "woods of ypres", "woody guthrie", "woody's a girl", "world party", "worm is green", "wormrot", "wouter hamel", "wovenhand", "wreck and reference", "wreckshop family", "write this down", "wu-tang clan", "wumpscut", "wyclef jean", "wynardtage", "wynn stewart", "wynonna judd", "wynter gordon", "wyrd", "x", "x japan", "x-fusion", "x-perience", "xandria", "xentrifuge", "xentrix", "xiu xiu", "xlooking forwardx", "xp8", "xtc", "xv", "xxxtentacion", "xzibit", "y&t", "y'akoto", "y-o-u", "yacht", "yank rachell", "yann tiersen", "yazoo", "yeah yeah yeahs", "yearning", "years & years", "yeasayer", "yefim bronfman", "yehuda hanani", "yehudi menuhin", "yello", "yellowcard", "yellowman", "yelworc", "yendri", "yes", "yg", "yngwie malmsteen", "ynw melly", "yo la tengo", "yo yo ma", "yob", "yodelice", "yoko ono", "youn sun nah", "young dro", "young galaxy", "young guns", "young jeezy", "young mc", "young money", "young the giant", "young thug", "your demise", "yourcodenameis", "youri egorov", "youssou n'dour", "youth lagoon", "ytcracker", "yukmouth", "yung joc", "yung lean", "yvonne elliman", "z-ro", "zac brown band", "zachary richard", "zakk wylde", "zap mama", "zayn", "zebra", "zed", "zed yago", "zedd", "zendaya", "zero 7", "zeromancer", "zhu", "ziggy", "ziggy alberts", "ziggy marley", "ziggy marley & the melody makers", "zilch", "zino francescatti", "zita swoon", "zoegirl", "zola jesus", "zolof the rock & roll destroyer", "zombina and the skeletones", "zoot woman", "zucchero", "zuill bailey", "zwan", "zyklon", "zz top", "¡mayday!", "émilie simon", "έλενα παπαρίζου", "борис гребенщиков", "наив", "хелависа", "シド (sid)", "ナイトメア (nightmare)", "大塚愛 (ai otsuka)", "岡崎律子 (ritsuko okazaki)", "志方あきこ", "明星 (akeboshi)", "李玟 (coco lee)", "梶浦由記 (yuki kajiura)", "王力宏 (leehom wang)", "目黒将司 (shoji meguro)", "蔡依林 (jolin tsai)", "陰陽座"]
genre = "alternative rock" #@param [ "2-tone", "a cappella", "acid house", "acid jazz", "acid rock", "acid techno", "acoustic blues", "acoustic", "adult contemporary", "adult contemporary", "african blues", "african folk", "african", "afrobeat", "afropop", "aggrotech", "alternative country", "alternative dance", "alternative hip hop", "alternative metal", "alternative rock", "alternative rock", "ambient", "american folk", "americana", "anarcho-punk", "andean music", "anime", "anti-folk", "appalachian folk", "arena rock", "art pop", "art punk", "art rock", "atmospheric black metal", "australian folk", "australian hip hop", "austropop", "avant-garde metal", "avant-garde", "avant-garde", "avant-pop", "axé", "axé", "bachata", "baião", "bakersfield sound", "balearic beat", "ballad", "banda", "barbershop", "baroque pop", "baroque", "beat", "beatbox", "bebop", "bhangra", "big band", "big beat", "black metal", "blackened death metal", "blackgaze", "blue-eyed soul", "bluegrass", "blues revival", "blues rock", "blues", "bolero", "boogie rock", "boogie woogie", "boogie", "bossa nova", "brazilian rock", "breakbeat", "breakcore", "brega", "british blues", "british invasion", "britpop", "bro-country", "brutal death metal", "bubblegum dance", "bubblegum pop", "cabaret", "cajun", "calypso", "canadian folk", "canadian hip hop", "canterbury", "cantopop", "celtic folk", "celtic fusion", "celtic punk", "celtic rock", "celtic", "chamber music", "chanson", "chicago blues", "children's music", "children's music", "chill-out", "chillwave", "chiptune", "choral", "christian hardcore", "christian hip hop", "christian hymns", "christian metal", "christian punk", "christian rock", "christian ska", "christian", "christmas", "classic female blues", "classic pop", "classic rock", "classical", "coldwave", "comedy rock", "comedy", "comic opera", "conjunto", "contemporary christian", "contemporary christian", "contemporary folk", "contemporary jazz", "contemporary r&b", "cool jazz", "corrido", "country blues", "country pop", "country pop", "country rap", "country rock", "country", "cowpunk", "crack rock steady", "crossover thrash", "crossover", "crunk", "crunkcore", "crust punk", "cumbia", "dance band", "dance punk", "dance", "dance-pop", "dance-pop", "dance-rock", "dancehall", "dark ambient", "dark cabaret", "dark electro", "dark rock", "dark wave", "dark wave‏", "death  metal", "death 'n' roll", "death doom", "death metal", "deathcore", "deathgrind", "deathrock", "deep house", "deep house", "delta blues", "detroit blues", "deutschpunk", "deutschrock", "digital hardcore", "dirty blues", "disco", "disney", "dixieland", "djent", "doo-wop", "doom metal", "downtempo", "dream pop", "drone doom", "drone music", "drum and bass", "drumstep", "dub", "dubstep", "duranguense", "early music", "east coast blues", "east coast hip hop", "easy listening", "ebm", "edm", "electric blues", "electric blues", "electric folk", "electro house", "electro swing", "electro", "electro-industrial", "electronic rock", "electronic", "electronica", "electronicore", "electropop", "electropop", "electropunk", "emo", "english folk", "enka", "ethereal wave", "euro disco", "eurobeat", "eurodance", "europop", "event", "exotica", "experimental hip hop", "experimental pop", "experimental rock", "experimental", "extreme metal", "fado", "filk", "finnish folk", "flamenco", "folk metal", "folk punk", "folk rock", "folk", "folktronica", "forró", "francophone", "freak folk", "free jazz", "freestyle", "french hip hop", "french house", "french pop", "funeral doom", "funk carioca", "funk melody", "funk metal", "funk rock", "funk", "funky house", "future garage", "futurepop", "g-funk", "gabber", "gangsta rap", "garage punk", "garage rock", "glam metal", "glam rock", "glitch hop", "glitch", "goa trance", "goregrind", "gospel blues", "gospel", "gothic metal", "gothic rock", "gothic", "gregorian chant", "grime", "grindcore", "groove metal", "grunge", "grupera", "gypsy jazz", "gypsy punk", "gypsy", "hamburger schule", "happy hardcore", "hard bop", "hard rock", "hard trance", "hardcore hip hop", "hardcore punk", "hardstyle", "harmonica blues", "hatecore", "hawaiian", "heartland rock", "heavy metal", "hi-nrg", "hill country blues", "hip hop", "hip house", "honky tonk", "horror punk", "horror punk‏", "horrorcore", "house", "humppa", "humppa", "idm", "indian classical music", "indie folk", "indie pop", "indie rock", "indietronica", "industrial metal", "industrial metal", "industrial rock", "industrial", "instrumental rock", "irish folk", "italian folk", "italo disco", "j-pop", "j-rap", "j-rock", "jam band", "jangle pop", "jazz blues", "jazz fusion", "jazz fusion", "jazz", "jazz", "jazz-funk", "jovem guarda", "jug band", "jump blues", "junkanoo", "k-pop", "kirtan", "kizomba", "klezmer", "krautrock", "la confusion des genres", "latin hip hop", "latin jazz", "latin pop", "latin rock", "latin", "lo-fi", "louisiana blues", "lounge music", "mambo", "mandopop", "manele", "mariachi", "martial industrial", "mashup", "math rock", "mathcore", "mbaqanga", "medieval", "melodic death metal", "melodic hardcore", "melodic metalcore", "memphis blues", "metalcore", "miami bass", "middle-eastern hip hop", "middle-eastern", "midwest rap", "minimal techno", "minimal", "minnesang", "mixed", "mod revival", "modern classical", "modern classical", "morna", "movie", "mpb", "musical", "mystic folk", "n", "nasheed", "native american", "nazi-punk", "nederpop", "neo soul", "neo-progressive rock", "neo-psychedelia", "neo-psychedelia‏", "neoclassical", "neofolk", "nerdcore hip hop", "neue deutsche härte", "neue deutsche welle", "new age", "new jack swing", "new orleans blues", "new orleans rhythm and blues", "new rave", "new wave", "nintendocore", "noise pop", "noise rock", "noise", "norteño", "novelty", "nsbm", "nu jazz", "nu metal", "nu metalcore", "nu-disco", "nwobhm", "oi-punk", "old-time music", "opera", "operatic pop", "orchestral", "outlaw country", "outsider music", "pagan metal", "paisley underground", "parody", "persian", "piano blues", "piano rock", "piedmont blues", "pirate metal", "poetry", "political folk", "political hip hop", "polka", "pop punk", "pop rock", "pop rock", "pop", "pop-folk", "pop-rap", "post-bop", "post-britpop", "post-grunge", "post-hardcore", "post-hardcore", "post-industrial", "post-metal", "post-punk revival", "post-punk", "post-punk", "post-rock", "post-rock", "power metal", "power noise", "power pop", "powerviolence", "pre-war blues", "progg", "progressive  rock", "progressive black metal", "progressive bluegrass", "progressive country", "progressive death metal", "progressive folk", "progressive house", "progressive metal", "progressive metalcore", "progressive rock", "progressive trance", "proto-punk", "psychedelic folk", "psychedelic pop", "psychedelic rock", "psychedelic rock", "psychedelic trance", "psychedelic", "psychobilly", "pub rock", "punk blues", "punk cabaret", "punk rock", "queercore", "quiet storm", "r&b", "rac", "ragga", "ragtime", "ranchera", "rap  metal", "rap metal", "rap rock", "rapcore", "rave", "raï", "reggae fusion", "reggae rock", "reggae", "reggaeton", "reggae‏", "religious", "rio", "riot grrrl", "rock 'n' roll", "rock 'n' roll", "rock against communism", "rock en español", "rock opera", "rock", "rockabilly", "romantic", "roots reggae", "roots rock", "roots", "russian rock", "s music", "salsa", "samba", "schlager", "scottish folk", "screamo", "sea shanties", "sertanejo", "shibuya-kei", "shoegazing", "singer only", "singer-songwriter", "singer-songwriter", "ska punk", "ska punk", "ska", "skate punk", "skiffle", "slowcore", "sludge metal", "smooth jazz", "soca", "soft rock", "sophisti-pop", "soul blues", "soul jazz", "soul", "soundtrack musical", "soundtrack television", "soundtrack", "southern gospel", "southern hip hop", "southern metal", "southern rock", "space rock", "speed metal", "speedcore", "spirituals", "spoken word", "steampunk", "stoner rock", "street punk", "string band", "sufi", "sunshine pop", "surf rock", "surf", "swamp blues", "swamp pop", "swamp rock", "swing", "swiss rock", "symphonic black metal", "symphonic metal", "symphonic metal", "symphonic rock", "synthpop", "synthpop", "synthwave", "tango", "tech house", "technical death metal", "techno", "teen pop", "teen pop", "tejano", "television", "texas blues", "thrash metal", "thrashcore", "trad jazz", "traditional bluegrass", "traditional folk", "trance", "trap", "trip hop", "tropicalismo", "uk garage", "unblack metal", "underground hip  hop", "underground hip hop", "unknown", "urban contemporary", "vaudeville", "video game", "video game", "viking metal", "visual kei", "vocal jazz", "vocal trance", "vocal", "vocalese", "volkslied", "volksmusik", "west coast blues", "west coast hip hop", "western swing", "western", "work songs", "worldbeat", "worship", "zydeco","electroclash"] {type:"string"}


lyrics = "I must not fear. Fear is the mind-killer. Fear is the little-death that brings total obliteration. I will face my fear. I will permit it to pass over me and through me. And when it has gone past, I will turn the inner eye to see its path. Where the fear has gone there will be nothing. Only I will remain" #@param {"type":"string"}

metas = [dict(artist = artist,
            genre = genre,
            total_length = hps.sample_length,
            offset = 0,
            lyrics = lyrics,
            ),
          ] * hps.n_samples
labels = top_prior.labeller.get_batch_labels(metas, 'cuda')



try:
  print(f'Artista escolhido {artist}')
  print(f'Genero escolhido {genre}')
  file = open("./photos/" + artist + ".jpg" , "rb")
  image = file.read()
  imageW = widgets.Image(
    value=image,
    format='jpg',
    width=300,
    height=400
  )
  IPython.display.display(imageW)
except FileNotFoundError:
  print("Sem imagem disponível")


## Generando opções para o inicio da música

Aqui você pode alterar a quantidade de segundos que a sua primeira geração vai ter, lembrando que no modo **Primed**, vamos ter obrigatoriamente os 12 primeiros segundos com o audio original que foi feito o upload, logo:

*   **Segundos no modo Primed**: 12 segundos + initial_generation_in_seconds
*   **Segundos no modo Ancestral**: initial_generation_in_seconds

E também pode se mudar a temperatura das amostrar (fator de aletoridade), o mais recomendado é valores entre .98 e .995

-----
## Generating options for the beginning of the song
Here you can change the length of your first generation, please note that the **Primed** mode demands that the 12 first seconds will be the same as in the original audio file the we uploaded, therefore we have:
*   **Seconds in the Primed mode**: 12 seconds + initial_generation_in_seconds
*   **Seconds in the Ancestral mode**: initial_generation_in_seconds

You can also change the sample temperature (randomness factor). The most recommended are values between .98 and .995

In [ ]:
#@title 
def seconds_to_tokens(sec, sr, prior, chunk_size):
  tokens = sec * hps.sr // prior.raw_to_tokens
  tokens = ((tokens // chunk_size) + 1) * chunk_size
  assert tokens <= prior.n_ctx, 'Choose a shorter generation length to stay within the top prior context'
  return tokens

initial_generation_in_seconds =  4 #@param {type:"slider", min:4, max:23, step:1}
tokens_to_sample = seconds_to_tokens(initial_generation_in_seconds, hps.sr, top_prior, chunk_size)

sampling_temperature = 0.995 #@param {type:"slider", min:0.9, max:0.995, step:0.005}

lower_batch_size = 16
max_batch_size = 3 if model in ('5b', '5b_lyrics') else 16
lower_level_chunk_size = 32
chunk_size = 16 if model in ('5b', '5b_lyrics') else 32
sampling_kwargs = dict(temp=sampling_temperature, fp16=True, max_batch_size=lower_batch_size,
                        chunk_size=lower_level_chunk_size)

In [ ]:
#@title Comece o Treinamento #1 / Start Training #1
if sample_hps.mode == 'ancestral':
  zs=[t.zeros(hps.n_samples,0,dtype=t.long, device='cuda') for _ in range(3)]
  zs=sample_partial_window(zs, labels, sampling_kwargs, 2, top_prior, tokens_to_sample, hps)
elif sample_hps.mode == 'primed':
  assert sample_hps.audio_file is not None
  audio_files = sample_hps.audio_file.split(',')
  duration = (int(sample_hps.prompt_length_in_seconds*hps.sr)//top_prior.raw_to_tokens)*top_prior.raw_to_tokens
  x = load_prompts(audio_files, duration, hps)
  zs = top_prior.encode(x, start_level=0, end_level=len(priors), bs_chunks=x.shape[0])
  zs = sample_partial_window(zs, labels, sampling_kwargs, 2, top_prior, tokens_to_sample, hps)
x = vqvae.decode(zs[2:], start_level=2).cpu().numpy()

for i in range(hps.n_samples):
  librosa.output.write_wav(f'noisy_top_level_generation_{i}.wav', x[i], sr=44100)

Com o termino do seu primeiro treinamento, roda a célula abaixo para que apareça em cada janela seu aúdio gerado correspondente, depois de ouvir todas as músicas que foram geradas nesse primeiro treinamento, escolha apenas **uma** música no formúlario abaixo, para que ela seja a base para os próximos treinamentos. Se não gostar de nenhuma, pode rodar novamente a celula de treinamento para gerar outros exemplos.

** OBSERVAÇÃO: este aúdio ainda contêm muito ruído, faça upsample totalmente (na próxima seção) para ouvir a versão de áudio final.

** OBSERVAÇÃO: Quando for escolher qual música, escolha o número correspondente ao da janela, se a janela do tab, tiver com a numeração 5, então a sua escolha de música foi a 5.

** OBSERVAÇÃO: É possível fazer o Download do Aúdio, apenas clicando nos 3 pontinhos do player de aúdio e apertando em "Download".

-----
After the first training finishes, run the below cell so it will appear in each window their generated audio. After listening all the songs generated in this first training, choose only *one* song in the below form, so it'll be the base for the next trainings. If you don't like any, you can run the training cell again to generate other examples.

NOTE: this audio still is very noisy, upsample it totally (in the next section) to listen to the final audio version.

NOTE: when you choose the song, choose the number related to the window one. If the tab has the number 5, then your choise was audio number 5.

NOTE: it is possible to download the audio by just clicking on the kebab menu, and then pressing "Download".

In [ ]:
#@title Escolha uma música / Choose a song.

stringsMusics = []

for stringMusic in os.listdir():
  stringAux = stringMusic.split(".wav")[0]
  stringAux = stringAux.split("_")
  if stringAux[:4] == ['noisy', 'top', 'level', 'generation']:
    stringsMusics.append(stringAux[4])

tab = google.colab.widgets.TabBar(stringsMusics)

for idx in range(len(stringsMusics)):
  index = str(idx)
  with tab.output_to(idx):
    IPython.display.display(Audio("noisy_top_level_generation_"+index+".wav"))


my_choice=5  #@param {type:"integer"}

## Continuando para os próximos treinamentos da música escolhida

Vamos aqui validar se sua escolha da música foi feita de maneira correta e se o número existe. Também vamos habilitar o Checkpoint, facilitando para voltar a uma célula e quebrar o treinamento em partes.

-----
## Following to the next trainings with the chosen song.

Let's validate if your audio choice was made properly and if the number exists. We'll also anable Checkpoint, making it easier to go back to a cell and break the training in parts.

In [ ]:
#@title Validação da música escolhida e Baixar CheckPoint #1 / Validate the chosen audio and download Checkpoint #1
if my_choice < len(stringsMusics) and my_choice >= 0:
  zs[2]=zs[2][my_choice].repeat(hps.n_samples,1)
  t.save(zs, 'zs-checkpoint2.t')
else:
  print("Escolha de número errado, por favor escolha entre o intervalo certo")




In [ ]:
#@title Carregar Checkpoint #1 / Load Checkpoint #1
# Set to True to load the previous checkpoint:
if False:
  zs=t.load('zs-checkpoint2.t') 

Escolha a duração da continuação. O modelo 1B pode gerar amostras de até 17 segundos e o 5B até 23 segundos, mas você vai querer escolher um comprimento de continuação mais curto para que seja capaz de olhar para trás o que você já gerou.

------
Choose the following sample length. 1B model can generate samples long up to 17 seconds and 5B can make it up to 23 seconds, but you'll want to choose a shorter length so it'll be able to look back to what you already generated.

In [ ]:
#@title Continuar geração / Continue generation
continue_generation_in_seconds=4 #@param {type:"slider", min:4, max:23, step:1}
tokens_to_sample = seconds_to_tokens(continue_generation_in_seconds, hps.sr, top_prior, chunk_size)

In [ ]:
#@title Treinamento Segunda Parte / Second part of training
zs = sample_partial_window(zs, labels, sampling_kwargs, 2, top_prior, tokens_to_sample, hps)
x = vqvae.decode(zs[2:], start_level=2).cpu().numpy()

Agora ouça as versões mais longas da amostra selecionada e escolha novamente uma amostra favorita. Se você não gostar de nenhum, volte para a célula onde você pode carregar o checkpoint e continue de lá.

Quando as amostras começam a ficar longas, você pode não querer ouvir desde o início, então altere a hora de início da reprodução mais tarde, se desejar.

---

Now listen to the longer versions of the sample you selected, and again choose a favorite sample.  If you don't like any, return back to the cell where you can load the checkpoint, and continue again from there.

When the samples start getting long, you might not always want to listen from the start, so change the playback start time later on if you like.

In [ ]:
playback_start_time_in_seconds = 0 

for i in range(hps.n_samples):
  librosa.output.write_wav(f'top_level_continuation_{i}.wav', x[i][playback_start_time_in_seconds*44100:], sr=44100)


stringsMusics = []

for stringMusic in os.listdir():
  stringAux = stringMusic.split(".wav")[0]
  stringAux = stringAux.split("_")
  if stringAux[:3] == ['top', 'level', 'continuation']:
    stringsMusics.append(stringAux[3])

tab = google.colab.widgets.TabBar(stringsMusics)

for idx in range(len(stringsMusics)):
  index = str(idx)
  with tab.output_to(idx):
    IPython.display.display(Audio("top_level_continuation_"+index+".wav"))


choice=0  #@param {type:"integer"}

# Faça Upsample da co-composição para maior qualidade de áudio

Escolhido novamente o outro aúdio que vamos utilizar para prosseguir o treinamento, agora vamos gerar músicas com uma qualidade de alto maior.

# Upsample Co-Composition to Higher Audio Quality

After we choose again the other audio that we'll use to proceed the training, now we'll generate song with a higher quality

In [ ]:
#@title Salvar checkpoint #2 / Save checkpoint #2
select_best_sample = True  # Set false if you want to upsample all your samples 
                           # upsampling sometimes yields subtly different results on multiple runs,
                           # so this way you can choose your favorite upsampling

if choice < len(stringsMusics) and choice >= 0:
  zs[2]=zs[2][choice].repeat(zs[2].shape[0],1)
  t.save(zs, 'zs-top-level-final.t')
else:
  print("Escolha de número errado, por favor escolha entre o intervalo certo")


In [ ]:
#@title Carregar checkpoint #2 / Load checkpoint #2
if True:
  zs = t.load('zs-top-level-final.t')

assert zs[2].shape[1]>=2048, f'Please first generate at least 2048 tokens at the top level, currently you have {zs[2].shape[1]}'
hps.sample_length = zs[2].shape[1]*top_prior.raw_to_tokens

Nota: Se você estiver usando um tempo de execução hospedado CoLab na camada gratuita, você pode querer baixar este arquivo zs-top-level-final.t e, em seguida, reiniciar uma instância e carregá-la na próxima célula. O nível gratuito dura no máximo 12 horas, e o estágio de upsampling pode levar muitas horas, dependendo de quanto tempo você gerou uma amostra.

---

Note: If you are using a CoLab hosted runtime on the free tier, you may want to download this zs-top-level-final.t file, and then restart an instance and load it in the next cell.  The free tier will last a maximum of 12 hours, and the upsampling stage can take many hours, depending on how long a sample you have generated.

In [ ]:
#@title Treinamento #3 / Training #3
# Set this False if you are on a local machine that has enough memory (this allows you to do the
# lyrics alignment visualization). For a hosted runtime, we'll need to go ahead and delete the top_prior
# if you are using the 5b_lyrics model.
if True:
  del top_prior
  empty_cache()
  top_prior=None

upsamplers = [make_prior(setup_hparams(prior, dict()), vqvae, 'cpu') for prior in priors[:-1]]

sampling_kwargs = [dict(temp=.99, fp16=True, max_batch_size=2, chunk_size=8),
                    dict(temp=0.99, fp16=True, max_batch_size=2, chunk_size=8),
                    None]

if type(labels)==dict:
  labels = [prior.labeller.get_batch_labels(metas, 'cuda') for prior in upsamplers] + [labels]

zs = upsample(zs, labels, sampling_kwargs, [*upsamplers, top_prior], hps) 

Esta próxima etapa aumenta a amostragem de 2 níveis. As amostras level_1 estarão disponíveis após cerca de uma hora (dependendo da duração da sua amostra) e são salvas em {hps.name} /level_0/item_0.wav, enquanto o level_0 totalmente ampliado provavelmente levará de 4 a 12 horas. Você pode acessar os arquivos wav abaixo, ou usando o painel "Arquivos" à esquerda deste CoLab.

(Observe, se você estiver usando este CoLab no nível gratuito do Google, você pode querer fazer o download das etapas intermediárias, pois a conexão durará no máximo 12 horas.)

----

This next step upsamples 2 levels. The level_1 samples will be available after around one hour (depending on the length of your sample) and are saved under {hps.name}/level_0/item_0.wav, while the fully upsampled level_0 will likely take 4-12 hours. You can access the wav files down below, or using the "Files" panel at the left of this CoLab.

(Please note, if you are using this CoLab on Google's free tier, you may want to download intermediate steps as the connection will last for a maximum 12 hours.)

In [ ]:
#@title Escute as Versões Finais De Aúdio Geradas / Listen to the final audio versions generated
stringMusic = []

for stringMusic in os.listdir(hps.name+"/level_0/"):
  stringAux = stringMusic.split(".wav")[0]
  stringAux = stringAux.split("_")
  if stringAux[:1] == ['item']:
    stringsMusics.append(stringAux[1])

tab = google.colab.widgets.TabBar(stringsMusics)

for idx in range(len(stringsMusics)):
  index = str(idx)
  with tab.output_to(idx):
    IPython.display.display(Audio(f'{hps.name}/level_0/item_{idx}.wav'))